# Ordering by metrics and retraining phase

## Dataset: MNIST

## Experiment configuration 3


In [ ]:
pip install --user tensorflow==2.5

In [1]:
import argparse

import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K

import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.regularizers import l2

import glob
import os
import cv2
import tensorflow.keras as keras


import time
import argparse

from tqdm import tqdm

from tensorflow.keras.models import load_model, Model


In [2]:
tf.__version__ == '2.5.0' # Version of tensorflow

True

In [3]:
pwd

'D:\\guided-retraining\\notebooks'

In [4]:
cd ../utils/

D:\guided-retraining\utils


In [5]:
# utils for project
#import utils_guided_retraining2 as utils
import utils_guided_retraining2 as utils

2.5.0
keras
2.5.0


In [6]:
#cd '../notebooks/'

In [7]:
dataset = "fashion"

In [8]:
x_train,y_train = utils.get_data(dataset,"Train",verbose=True)
x_val,y_val = utils.get_data(dataset,"Val",verbose=True)
x_test,y_test = utils.get_data(dataset,"Test",verbose=True)

D:/guided-retraining/data/fashion/x_train.npy
x_set len:  48999
D:/guided-retraining/data/fashion/y_train.npy
y_set len:  48999
D:/guided-retraining/data/fashion/x_val.npy
x_set len:  14000
D:/guided-retraining/data/fashion/y_val.npy
y_set len:  14000
D:/guided-retraining/data/fashion/x_test.npy
x_set len:  7001
D:/guided-retraining/data/fashion/y_test.npy
y_set len:  7001


In [9]:
x_train_and_adversary,y_train_and_adversary = utils.get_data(dataset,"Train_and_adversary",verbose=True)

D:/guided-retraining/data/data_adversarial_july/fashion/train_and_adversary.npy
x_set len:  55998
D:/guided-retraining/data/data_adversarial_july/fashion/train_and_adversary_labels.npy
y_set len:  55998


In [10]:
x_adversary_training = x_train_and_adversary[len(x_train):]
print(len(x_adversary_training))
y_adversary_training = y_train_and_adversary[len(y_train):]

print(len(y_adversary_training))

6999
6999


In [11]:
# Obtaining adversarial examples for testing 
x_test_and_adversary,y_test_and_adversary = utils.get_adversarial_data(dataset,'Test_adversarial',True) 

D:/guided-retraining/data/data_adversarial_july/fashion/test_and_adversary.npy D:/guided-retraining/data/data_adversarial_july/fashion/test_and_adversary_labels.npy
D:/guided-retraining/data/data_adversarial_july/fashion/test_and_adversary.npy
x_set len:  14000
D:/guided-retraining/data/data_adversarial_july/fashion/test_and_adversary_labels.npy
y_set len:  14000


In [12]:
x_adversary_test = x_test_and_adversary[len(x_test):]
print(len(x_adversary_test))
y_adversary_test = y_test_and_adversary[len(y_test):]

print(len(y_adversary_test))

6999
6999


## ----

In [13]:
# Original model 

model_dir = "C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06/"
if(dataset == 'gtsrb'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06"
elif(dataset == 'intel'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/intel_model_21_10"
elif(dataset == 'mnist'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/models2"
elif(dataset == 'cifar'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/model_02"
elif(dataset == 'fashion'):
    model_dir = "D:/guided-retraining/models/model_fashion_2"


In [14]:
print(model_dir)
model_original = utils.My_model(dataset,True, model_dir)

D:/guided-retraining/models/model_fashion_2
Model loaded correctly


In [15]:
print(len(x_adversary_training))

6999


In [16]:
print(x_adversary_training.shape)
len(x_adversary_training)//20

(6999, 28, 28, 1)


349

In [17]:
n_data_points = 340

In [18]:
import gc

## Loading LSA and DSA values 

In [19]:
lsa_direction = "D:/guided-retraining/data/"+dataset+"/lsa_values.npy"


In [20]:
#lsa_direction = "C:/Users/fjdurlop/Documents/upc/upc-july/data/"+dataset+"/lsa_values.npy"


In [21]:
lsa_values = np.load(lsa_direction)[len(x_train):] 
print(len(lsa_values))


6999


In [22]:

# Obtaining top n images by LSA values
top_images_by_lsa = utils.get_x_of_indexes(list(np.flip(np.argsort(lsa_values))),x_adversary_training)
top_labels_by_lsa = utils.get_x_of_indexes(list(np.flip(np.argsort(lsa_values))),y_adversary_training)


In [23]:
m = n_data_points
n = 0
image_sets_lsa = []
label_sets_lsa = []

# last
#for i in range(0,len(top_images_by_lsa)//m):

for i in range((len(top_images_by_lsa)//m)):
    print(i,":")
    if (i+1 >= ((len(top_images_by_lsa)//m))):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_lsa)%m))
        top_images_by_lsa_n = np.array(top_images_by_lsa[:n+m+(len(top_images_by_lsa)%m)])
        top_labels_by_lsa_n = np.array(top_labels_by_lsa[:n+m+(len(top_images_by_lsa)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_lsa_n = np.array(top_images_by_lsa[:n+m])
        top_labels_by_lsa_n = np.array(top_labels_by_lsa[:n+m])
    image_sets_lsa.append(top_images_by_lsa_n)
    label_sets_lsa.append(top_labels_by_lsa_n)
    print(len(top_images_by_lsa_n))
    n += m

0 :
0  ->  340
340
1 :
0  ->  680
680
2 :
0  ->  1020
1020
3 :
0  ->  1360
1360
4 :
0  ->  1700
1700
5 :
0  ->  2040
2040
6 :
0  ->  2380
2380
7 :
0  ->  2720
2720
8 :
0  ->  3060
3060
9 :
0  ->  3400
3400
10 :
0  ->  3740
3740
11 :
0  ->  4080
4080
12 :
0  ->  4420
4420
13 :
0  ->  4760
4760
14 :
0  ->  5100
5100
15 :
0  ->  5440
5440
16 :
0  ->  5780
5780
17 :
0  ->  6120
6120
18 :
0  ->  6460
6460
19 :
Last
0  ->  6999
6999


In [24]:
len(image_sets_lsa[-1])

6999

## Training guided by LSA

In [25]:
len(label_sets_lsa)

20

In [26]:
print(model_dir)

D:/guided-retraining/models/model_fashion_2


In [27]:
print(model_dir)

models_lsa = []
for i in range(len(label_sets_lsa)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_lsa.append(model)


D:/guided-retraining/models/model_fashion_2
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [28]:
models_lsa[0].evaluate(x_test,y_test)

219/219 [==============================] - 2s 8ms/step - loss: 0.2254 - accuracy: 0.9190


[0.2253975123167038, 0.9190115928649902]

In [29]:
models_lsa[0].evaluate(x_test_and_adversary,y_test_and_adversary)

438/438 [==============================] - 3s 7ms/step - loss: 0.5032 - accuracy: 0.8342


[0.5032416582107544, 0.8342142701148987]

In [30]:
n = 0
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n=n+1

Epoch 1/10
6/6 [==============================] - 3s 562ms/step - loss: 1.3568 - accuracy: 0.5500 - val_loss: 0.2490 - val_accuracy: 0.9096
Epoch 2/10
6/6 [==============================] - 3s 535ms/step - loss: 0.8383 - accuracy: 0.7147 - val_loss: 0.2768 - val_accuracy: 0.9016
Epoch 3/10
6/6 [==============================] - 3s 522ms/step - loss: 0.5518 - accuracy: 0.8029 - val_loss: 0.2842 - val_accuracy: 0.9009
Epoch 4/10
6/6 [==============================] - 3s 522ms/step - loss: 0.5022 - accuracy: 0.8206 - val_loss: 0.2922 - val_accuracy: 0.8969
Epoch 5/10
6/6 [==============================] - 3s 602ms/step - loss: 0.4368 - accuracy: 0.8441 - val_loss: 0.2993 - val_accuracy: 0.8949
Epoch 6/10
6/6 [==============================] - 3s 610ms/step - loss: 0.3806 - accuracy: 0.8618 - val_loss: 0.3027 - val_accuracy: 0.8953
Epoch 7/10
6/6 [==============================] - 3s 661ms/step - loss: 0.3461 - accuracy: 0.8971 - val_loss: 0.3115 - val_accuracy: 0.8910
Epoch 8/10
6/6 [====

In [31]:
n=1
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

Epoch 1/10
11/11 [==============================] - 4s 379ms/step - loss: 1.1050 - accuracy: 0.6485 - val_loss: 0.2729 - val_accuracy: 0.8994
Epoch 2/10
11/11 [==============================] - 4s 351ms/step - loss: 0.7583 - accuracy: 0.7632 - val_loss: 0.2810 - val_accuracy: 0.8966
Epoch 3/10
11/11 [==============================] - 3s 333ms/step - loss: 0.6103 - accuracy: 0.7912 - val_loss: 0.2882 - val_accuracy: 0.8925
Epoch 4/10
11/11 [==============================] - 4s 359ms/step - loss: 0.5423 - accuracy: 0.8250 - val_loss: 0.2996 - val_accuracy: 0.8874
Epoch 5/10
11/11 [==============================] - 3s 340ms/step - loss: 0.4902 - accuracy: 0.8456 - val_loss: 0.2959 - val_accuracy: 0.8892
Epoch 6/10
11/11 [==============================] - 4s 343ms/step - loss: 0.3965 - accuracy: 0.8544 - val_loss: 0.3132 - val_accuracy: 0.8858
Epoch 7/10
11/11 [==============================] - 3s 335ms/step - loss: 0.3819 - accuracy: 0.8809 - val_loss: 0.3128 - val_accuracy: 0.8880
Epoch 

In [32]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

2
Epoch 1/10
16/16 [==============================] - 4s 254ms/step - loss: 1.0377 - accuracy: 0.6833 - val_loss: 0.2835 - val_accuracy: 0.8946
Epoch 2/10
16/16 [==============================] - 4s 250ms/step - loss: 0.7162 - accuracy: 0.7637 - val_loss: 0.2969 - val_accuracy: 0.8884
Epoch 3/10
16/16 [==============================] - 4s 244ms/step - loss: 0.5703 - accuracy: 0.8098 - val_loss: 0.3001 - val_accuracy: 0.8838
Epoch 4/10
16/16 [==============================] - 4s 239ms/step - loss: 0.5006 - accuracy: 0.8343 - val_loss: 0.3016 - val_accuracy: 0.8857
Epoch 5/10
16/16 [==============================] - 5s 318ms/step - loss: 0.4166 - accuracy: 0.8490 - val_loss: 0.3202 - val_accuracy: 0.8798
Epoch 6/10
16/16 [==============================] - 4s 279ms/step - loss: 0.3835 - accuracy: 0.8725 - val_loss: 0.3215 - val_accuracy: 0.8791
Epoch 7/10
16/16 [==============================] - 4s 263ms/step - loss: 0.3965 - accuracy: 0.8716 - val_loss: 0.3108 - val_accuracy: 0.8865
Epoc

In [33]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

3
Epoch 1/10
22/22 [==============================] - 5s 190ms/step - loss: 0.7879 - accuracy: 0.7566 - val_loss: 0.3002 - val_accuracy: 0.8832
Epoch 2/10
22/22 [==============================] - 4s 183ms/step - loss: 0.5575 - accuracy: 0.8007 - val_loss: 0.2971 - val_accuracy: 0.8853
Epoch 3/10
22/22 [==============================] - 4s 183ms/step - loss: 0.4584 - accuracy: 0.8412 - val_loss: 0.3023 - val_accuracy: 0.8841
Epoch 4/10
22/22 [==============================] - 4s 178ms/step - loss: 0.3922 - accuracy: 0.8699 - val_loss: 0.3186 - val_accuracy: 0.8789
Epoch 5/10
22/22 [==============================] - 4s 186ms/step - loss: 0.4018 - accuracy: 0.8684 - val_loss: 0.3135 - val_accuracy: 0.8821
Epoch 6/10
22/22 [==============================] - 4s 200ms/step - loss: 0.3331 - accuracy: 0.8897 - val_loss: 0.3125 - val_accuracy: 0.8864
Epoch 7/10
22/22 [==============================] - 4s 197ms/step - loss: 0.3099 - accuracy: 0.9015 - val_loss: 0.3273 - val_accuracy: 0.8821
Epoc

In [34]:
print(n) #4
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

4
Epoch 1/10
27/27 [==============================] - 5s 165ms/step - loss: 0.7198 - accuracy: 0.7724 - val_loss: 0.2885 - val_accuracy: 0.8893
Epoch 2/10
27/27 [==============================] - 4s 163ms/step - loss: 0.4789 - accuracy: 0.8382 - val_loss: 0.3142 - val_accuracy: 0.8785
Epoch 3/10
27/27 [==============================] - 4s 162ms/step - loss: 0.3856 - accuracy: 0.8729 - val_loss: 0.3169 - val_accuracy: 0.8792
Epoch 4/10
27/27 [==============================] - 4s 159ms/step - loss: 0.3271 - accuracy: 0.8912 - val_loss: 0.3315 - val_accuracy: 0.8769
Epoch 5/10
27/27 [==============================] - 4s 162ms/step - loss: 0.3232 - accuracy: 0.8882 - val_loss: 0.3002 - val_accuracy: 0.8860
Epoch 6/10
27/27 [==============================] - 4s 161ms/step - loss: 0.2749 - accuracy: 0.9012 - val_loss: 0.3224 - val_accuracy: 0.8819
Epoch 7/10
27/27 [==============================] - 4s 169ms/step - loss: 0.2601 - accuracy: 0.9088 - val_loss: 0.3298 - val_accuracy: 0.8794
Epoc

In [35]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

5
Epoch 1/10
32/32 [==============================] - 5s 153ms/step - loss: 0.6409 - accuracy: 0.8010 - val_loss: 0.2910 - val_accuracy: 0.8931
Epoch 2/10
32/32 [==============================] - 5s 151ms/step - loss: 0.4295 - accuracy: 0.8618 - val_loss: 0.2852 - val_accuracy: 0.8868
Epoch 3/10
32/32 [==============================] - 4s 143ms/step - loss: 0.3784 - accuracy: 0.8725 - val_loss: 0.3095 - val_accuracy: 0.8826
Epoch 4/10
32/32 [==============================] - 4s 139ms/step - loss: 0.3230 - accuracy: 0.8946 - val_loss: 0.3133 - val_accuracy: 0.8804
Epoch 5/10
32/32 [==============================] - 5s 149ms/step - loss: 0.2876 - accuracy: 0.9034 - val_loss: 0.3058 - val_accuracy: 0.8845
Epoch 6/10
32/32 [==============================] - 4s 142ms/step - loss: 0.2712 - accuracy: 0.9103 - val_loss: 0.3026 - val_accuracy: 0.8854
Epoch 7/10
32/32 [==============================] - 5s 144ms/step - loss: 0.2332 - accuracy: 0.9211 - val_loss: 0.3124 - val_accuracy: 0.8844
Epoc

In [36]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

6
Epoch 1/10
38/38 [==============================] - 5s 129ms/step - loss: 0.6478 - accuracy: 0.7899 - val_loss: 0.3051 - val_accuracy: 0.8876
Epoch 2/10
38/38 [==============================] - 5s 133ms/step - loss: 0.4705 - accuracy: 0.8378 - val_loss: 0.3016 - val_accuracy: 0.8872
Epoch 3/10
38/38 [==============================] - 5s 125ms/step - loss: 0.3843 - accuracy: 0.8626 - val_loss: 0.3070 - val_accuracy: 0.8829
Epoch 4/10
38/38 [==============================] - 5s 130ms/step - loss: 0.3271 - accuracy: 0.8870 - val_loss: 0.3149 - val_accuracy: 0.8811
Epoch 5/10
38/38 [==============================] - 5s 129ms/step - loss: 0.3037 - accuracy: 0.8945 - val_loss: 0.3039 - val_accuracy: 0.8843
Epoch 6/10
38/38 [==============================] - 5s 127ms/step - loss: 0.2931 - accuracy: 0.8983 - val_loss: 0.3299 - val_accuracy: 0.8816
Epoch 7/10
38/38 [==============================] - 5s 127ms/step - loss: 0.2652 - accuracy: 0.9113 - val_loss: 0.3156 - val_accuracy: 0.8824
Epoc

In [37]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

7
Epoch 1/10
43/43 [==============================] - 6s 122ms/step - loss: 0.6263 - accuracy: 0.7941 - val_loss: 0.3136 - val_accuracy: 0.8839
Epoch 2/10
43/43 [==============================] - 5s 122ms/step - loss: 0.4579 - accuracy: 0.8445 - val_loss: 0.3053 - val_accuracy: 0.8849
Epoch 3/10
43/43 [==============================] - 5s 121ms/step - loss: 0.3928 - accuracy: 0.8647 - val_loss: 0.3029 - val_accuracy: 0.8862
Epoch 4/10
43/43 [==============================] - 5s 122ms/step - loss: 0.3598 - accuracy: 0.8676 - val_loss: 0.3383 - val_accuracy: 0.8771
Epoch 5/10
43/43 [==============================] - 5s 128ms/step - loss: 0.3273 - accuracy: 0.8938 - val_loss: 0.3352 - val_accuracy: 0.8778
Epoch 6/10
43/43 [==============================] - 5s 127ms/step - loss: 0.3098 - accuracy: 0.8938 - val_loss: 0.3213 - val_accuracy: 0.8825
Epoch 7/10
43/43 [==============================] - 5s 126ms/step - loss: 0.2886 - accuracy: 0.9004 - val_loss: 0.3184 - val_accuracy: 0.8833
Epoc

In [38]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

8
Epoch 1/10
48/48 [==============================] - 6s 117ms/step - loss: 0.6205 - accuracy: 0.8072 - val_loss: 0.2840 - val_accuracy: 0.8966
Epoch 2/10
48/48 [==============================] - 5s 114ms/step - loss: 0.4417 - accuracy: 0.8493 - val_loss: 0.2920 - val_accuracy: 0.8914
Epoch 3/10
48/48 [==============================] - 5s 114ms/step - loss: 0.3722 - accuracy: 0.8699 - val_loss: 0.3085 - val_accuracy: 0.8854
Epoch 4/10
48/48 [==============================] - 5s 115ms/step - loss: 0.3441 - accuracy: 0.8830 - val_loss: 0.3098 - val_accuracy: 0.8845
Epoch 5/10
48/48 [==============================] - 6s 120ms/step - loss: 0.3085 - accuracy: 0.8980 - val_loss: 0.3133 - val_accuracy: 0.8877
Epoch 6/10
48/48 [==============================] - 6s 121ms/step - loss: 0.2971 - accuracy: 0.9029 - val_loss: 0.3121 - val_accuracy: 0.8882
Epoch 7/10
48/48 [==============================] - 6s 116ms/step - loss: 0.2683 - accuracy: 0.9075 - val_loss: 0.3020 - val_accuracy: 0.8904
Epoc

In [39]:
#
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

9
Epoch 1/10
54/54 [==============================] - 7s 112ms/step - loss: 0.5558 - accuracy: 0.8212 - val_loss: 0.3150 - val_accuracy: 0.8873
Epoch 2/10
54/54 [==============================] - 6s 113ms/step - loss: 0.4243 - accuracy: 0.8624 - val_loss: 0.3100 - val_accuracy: 0.8834
Epoch 3/10
54/54 [==============================] - 6s 111ms/step - loss: 0.3667 - accuracy: 0.8785 - val_loss: 0.3160 - val_accuracy: 0.8841
Epoch 4/10
54/54 [==============================] - 6s 111ms/step - loss: 0.3292 - accuracy: 0.8915 - val_loss: 0.3140 - val_accuracy: 0.8864
Epoch 5/10
54/54 [==============================] - 6s 109ms/step - loss: 0.3069 - accuracy: 0.8944 - val_loss: 0.3177 - val_accuracy: 0.8826
Epoch 6/10
54/54 [==============================] - 6s 112ms/step - loss: 0.3009 - accuracy: 0.9012 - val_loss: 0.3156 - val_accuracy: 0.8851
Epoch 7/10
54/54 [==============================] - 6s 109ms/step - loss: 0.2663 - accuracy: 0.9088 - val_loss: 0.3235 - val_accuracy: 0.8873
Epoc

In [40]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

10
Epoch 1/10
59/59 [==============================] - 8s 118ms/step - loss: 0.5217 - accuracy: 0.8286 - val_loss: 0.2860 - val_accuracy: 0.8892
Epoch 2/10
59/59 [==============================] - 6s 105ms/step - loss: 0.3851 - accuracy: 0.8733 - val_loss: 0.3119 - val_accuracy: 0.8812
Epoch 3/10
59/59 [==============================] - 6s 111ms/step - loss: 0.3572 - accuracy: 0.8845 - val_loss: 0.3141 - val_accuracy: 0.8836
Epoch 4/10
59/59 [==============================] - 7s 115ms/step - loss: 0.3201 - accuracy: 0.8944 - val_loss: 0.3048 - val_accuracy: 0.8880
Epoch 5/10
59/59 [==============================] - 6s 107ms/step - loss: 0.2970 - accuracy: 0.8995 - val_loss: 0.3044 - val_accuracy: 0.8874
Epoch 6/10
59/59 [==============================] - 6s 109ms/step - loss: 0.2702 - accuracy: 0.9083 - val_loss: 0.3234 - val_accuracy: 0.8885
Epoch 7/10
59/59 [==============================] - 6s 106ms/step - loss: 0.2670 - accuracy: 0.9155 - val_loss: 0.3237 - val_accuracy: 0.8866
Epo

In [41]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

11
Epoch 1/10
64/64 [==============================] - 7s 106ms/step - loss: 0.5183 - accuracy: 0.8284 - val_loss: 0.3070 - val_accuracy: 0.8845
Epoch 2/10
64/64 [==============================] - 7s 109ms/step - loss: 0.4002 - accuracy: 0.8635 - val_loss: 0.3152 - val_accuracy: 0.8849
Epoch 3/10
64/64 [==============================] - 7s 105ms/step - loss: 0.3433 - accuracy: 0.8853 - val_loss: 0.3217 - val_accuracy: 0.8841
Epoch 4/10
64/64 [==============================] - 7s 106ms/step - loss: 0.3069 - accuracy: 0.8931 - val_loss: 0.3289 - val_accuracy: 0.8851
Epoch 5/10
64/64 [==============================] - 7s 109ms/step - loss: 0.2891 - accuracy: 0.9054 - val_loss: 0.3220 - val_accuracy: 0.8805
Epoch 6/10
64/64 [==============================] - 7s 110ms/step - loss: 0.2556 - accuracy: 0.9135 - val_loss: 0.3196 - val_accuracy: 0.8849
Epoch 7/10
64/64 [==============================] - 7s 107ms/step - loss: 0.2465 - accuracy: 0.9191 - val_loss: 0.3162 - val_accuracy: 0.8887
Epo

In [42]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

12
Epoch 1/10
70/70 [==============================] - 8s 104ms/step - loss: 0.5448 - accuracy: 0.8115 - val_loss: 0.3151 - val_accuracy: 0.8824
Epoch 2/10
70/70 [==============================] - 7s 103ms/step - loss: 0.4237 - accuracy: 0.8532 - val_loss: 0.3145 - val_accuracy: 0.8797
Epoch 3/10
70/70 [==============================] - 7s 102ms/step - loss: 0.3642 - accuracy: 0.8695 - val_loss: 0.3109 - val_accuracy: 0.8853
Epoch 4/10
70/70 [==============================] - 7s 102ms/step - loss: 0.3322 - accuracy: 0.8824 - val_loss: 0.3425 - val_accuracy: 0.8799
Epoch 5/10
70/70 [==============================] - 7s 103ms/step - loss: 0.3168 - accuracy: 0.8966 - val_loss: 0.3361 - val_accuracy: 0.8794
Epoch 6/10
70/70 [==============================] - 8s 112ms/step - loss: 0.3018 - accuracy: 0.8962 - val_loss: 0.3191 - val_accuracy: 0.8857
Epoch 7/10
70/70 [==============================] - 7s 105ms/step - loss: 0.2567 - accuracy: 0.9149 - val_loss: 0.3444 - val_accuracy: 0.8821
Epo

In [43]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

13
Epoch 1/10
75/75 [==============================] - 8s 98ms/step - loss: 0.5426 - accuracy: 0.8174 - val_loss: 0.3080 - val_accuracy: 0.8793
Epoch 2/10
75/75 [==============================] - 7s 100ms/step - loss: 0.4409 - accuracy: 0.8477 - val_loss: 0.3038 - val_accuracy: 0.8850
Epoch 3/10
75/75 [==============================] - 7s 98ms/step - loss: 0.3918 - accuracy: 0.8626 - val_loss: 0.3163 - val_accuracy: 0.8824
Epoch 4/10
75/75 [==============================] - 7s 99ms/step - loss: 0.3497 - accuracy: 0.8775 - val_loss: 0.2993 - val_accuracy: 0.8892
Epoch 5/10
75/75 [==============================] - 7s 99ms/step - loss: 0.3187 - accuracy: 0.8937 - val_loss: 0.3194 - val_accuracy: 0.8870
Epoch 6/10
75/75 [==============================] - 7s 99ms/step - loss: 0.3018 - accuracy: 0.8956 - val_loss: 0.3149 - val_accuracy: 0.8852
Epoch 7/10
75/75 [==============================] - 7s 100ms/step - loss: 0.2837 - accuracy: 0.9034 - val_loss: 0.3255 - val_accuracy: 0.8874
Epoch 8/

In [44]:
#
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

14
Epoch 1/10
80/80 [==============================] - 9s 100ms/step - loss: 0.5823 - accuracy: 0.7971 - val_loss: 0.2843 - val_accuracy: 0.8888
Epoch 2/10
80/80 [==============================] - 8s 100ms/step - loss: 0.4532 - accuracy: 0.8394 - val_loss: 0.3072 - val_accuracy: 0.8814
Epoch 3/10
80/80 [==============================] - 8s 98ms/step - loss: 0.4007 - accuracy: 0.8549 - val_loss: 0.2902 - val_accuracy: 0.8898
Epoch 4/10
80/80 [==============================] - 8s 100ms/step - loss: 0.3693 - accuracy: 0.8741 - val_loss: 0.2932 - val_accuracy: 0.8889
Epoch 5/10
80/80 [==============================] - 8s 101ms/step - loss: 0.3453 - accuracy: 0.8839 - val_loss: 0.3077 - val_accuracy: 0.8904
Epoch 6/10
80/80 [==============================] - 8s 101ms/step - loss: 0.3113 - accuracy: 0.8927 - val_loss: 0.3017 - val_accuracy: 0.8895
Epoch 7/10
80/80 [==============================] - 8s 102ms/step - loss: 0.2963 - accuracy: 0.8990 - val_loss: 0.3053 - val_accuracy: 0.8936
Epoc

In [45]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

15
Epoch 1/10
85/85 [==============================] - 9s 103ms/step - loss: 0.5885 - accuracy: 0.7923 - val_loss: 0.2863 - val_accuracy: 0.8896
Epoch 2/10
85/85 [==============================] - 8s 101ms/step - loss: 0.4672 - accuracy: 0.8313 - val_loss: 0.2861 - val_accuracy: 0.8901
Epoch 3/10
85/85 [==============================] - 9s 111ms/step - loss: 0.4175 - accuracy: 0.8540 - val_loss: 0.2804 - val_accuracy: 0.8941
Epoch 4/10
85/85 [==============================] - 9s 102ms/step - loss: 0.3968 - accuracy: 0.8614 - val_loss: 0.2907 - val_accuracy: 0.8936
Epoch 5/10
85/85 [==============================] - 8s 100ms/step - loss: 0.3489 - accuracy: 0.8790 - val_loss: 0.2862 - val_accuracy: 0.8956
Epoch 6/10
85/85 [==============================] - 8s 99ms/step - loss: 0.3407 - accuracy: 0.8877 - val_loss: 0.2878 - val_accuracy: 0.8950
Epoch 7/10
85/85 [==============================] - 8s 100ms/step - loss: 0.3025 - accuracy: 0.9000 - val_loss: 0.2970 - val_accuracy: 0.8986
Epoc

In [46]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

16
Epoch 1/10
91/91 [==============================] - 10s 98ms/step - loss: 0.5945 - accuracy: 0.7888 - val_loss: 0.2802 - val_accuracy: 0.8976
Epoch 2/10
91/91 [==============================] - 9s 96ms/step - loss: 0.4944 - accuracy: 0.8244 - val_loss: 0.2846 - val_accuracy: 0.8949
Epoch 3/10
91/91 [==============================] - 9s 99ms/step - loss: 0.4339 - accuracy: 0.8439 - val_loss: 0.2768 - val_accuracy: 0.8975
Epoch 4/10
91/91 [==============================] - 9s 97ms/step - loss: 0.4095 - accuracy: 0.8583 - val_loss: 0.2842 - val_accuracy: 0.8948
Epoch 5/10
91/91 [==============================] - 9s 97ms/step - loss: 0.3757 - accuracy: 0.8682 - val_loss: 0.2885 - val_accuracy: 0.8978
Epoch 6/10
91/91 [==============================] - 9s 97ms/step - loss: 0.3640 - accuracy: 0.8772 - val_loss: 0.2869 - val_accuracy: 0.9004
Epoch 7/10
91/91 [==============================] - 9s 97ms/step - loss: 0.3108 - accuracy: 0.8865 - val_loss: 0.2923 - val_accuracy: 0.8960
Epoch 8/1

In [47]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

17
Epoch 1/10
96/96 [==============================] - 10s 96ms/step - loss: 0.6138 - accuracy: 0.7827 - val_loss: 0.2790 - val_accuracy: 0.8903
Epoch 2/10
96/96 [==============================] - 9s 95ms/step - loss: 0.4867 - accuracy: 0.8206 - val_loss: 0.2942 - val_accuracy: 0.8913
Epoch 3/10
96/96 [==============================] - 9s 98ms/step - loss: 0.4428 - accuracy: 0.8355 - val_loss: 0.2845 - val_accuracy: 0.8961
Epoch 4/10
96/96 [==============================] - 9s 96ms/step - loss: 0.4105 - accuracy: 0.8500 - val_loss: 0.2914 - val_accuracy: 0.8949
Epoch 5/10
96/96 [==============================] - 9s 95ms/step - loss: 0.3685 - accuracy: 0.8673 - val_loss: 0.2807 - val_accuracy: 0.8984
Epoch 6/10
96/96 [==============================] - 9s 95ms/step - loss: 0.3485 - accuracy: 0.8776 - val_loss: 0.3041 - val_accuracy: 0.8961
Epoch 7/10
96/96 [==============================] - 9s 94ms/step - loss: 0.3315 - accuracy: 0.8859 - val_loss: 0.2932 - val_accuracy: 0.8999
Epoch 8/1

In [48]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

18
Epoch 1/10
101/101 [==============================] - 10s 94ms/step - loss: 0.5947 - accuracy: 0.7839 - val_loss: 0.2784 - val_accuracy: 0.8934
Epoch 2/10
101/101 [==============================] - 9s 93ms/step - loss: 0.4790 - accuracy: 0.8238 - val_loss: 0.2723 - val_accuracy: 0.8964
Epoch 3/10
101/101 [==============================] - 9s 93ms/step - loss: 0.4376 - accuracy: 0.8423 - val_loss: 0.2725 - val_accuracy: 0.8977
Epoch 4/10
101/101 [==============================] - 9s 94ms/step - loss: 0.4230 - accuracy: 0.8560 - val_loss: 0.2795 - val_accuracy: 0.8971
Epoch 5/10
101/101 [==============================] - 9s 94ms/step - loss: 0.3811 - accuracy: 0.8633 - val_loss: 0.2828 - val_accuracy: 0.8990
Epoch 6/10
101/101 [==============================] - 9s 93ms/step - loss: 0.3430 - accuracy: 0.8802 - val_loss: 0.2986 - val_accuracy: 0.8955
Epoch 7/10
101/101 [==============================] - 9s 93ms/step - loss: 0.3111 - accuracy: 0.8916 - val_loss: 0.2886 - val_accuracy: 0.

In [49]:
#
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

19
Epoch 1/10
110/110 [==============================] - 11s 92ms/step - loss: 0.5514 - accuracy: 0.8060 - val_loss: 0.2649 - val_accuracy: 0.9013
Epoch 2/10
110/110 [==============================] - 10s 91ms/step - loss: 0.4708 - accuracy: 0.8361 - val_loss: 0.2735 - val_accuracy: 0.8984
Epoch 3/10
110/110 [==============================] - 10s 91ms/step - loss: 0.4158 - accuracy: 0.8491 - val_loss: 0.2875 - val_accuracy: 0.8950
Epoch 4/10
110/110 [==============================] - 10s 91ms/step - loss: 0.3923 - accuracy: 0.8638 - val_loss: 0.2955 - val_accuracy: 0.8929
Epoch 5/10
110/110 [==============================] - 10s 94ms/step - loss: 0.3612 - accuracy: 0.8740 - val_loss: 0.2898 - val_accuracy: 0.8989
Epoch 6/10
110/110 [==============================] - 11s 100ms/step - loss: 0.3218 - accuracy: 0.8873 - val_loss: 0.3031 - val_accuracy: 0.8946
Epoch 7/10
110/110 [==============================] - 11s 102ms/step - loss: 0.3064 - accuracy: 0.8946 - val_loss: 0.3083 - val_accu

In [50]:
#new_model_lsa_dir  = "D:/models/gtsrb_models/C3/gtsrb_model_c3_sep_lsa_e1"
new_model_lsa_dir  = "D:/models/aug_22/"+dataset+"/C3/"+dataset+"_model_c3_may_lsa_e1"
#new_model_lsa_dir  = "C:/Users/fjdurlop/Documents/upc/models/"+dataset+"/C3/"+dataset+"_model_c3_aug_lsa_e1"

i=0

for model in models_lsa:
    model.save(new_model_lsa_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_may_lsa_e1_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_may_lsa_e1_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_may_lsa_e1_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_may_lsa_e1_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_may_lsa_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_may_lsa_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_may_lsa_e1_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_may_lsa_e1_7\assets
Model has been saved
INFO:tensorflow:Assets written t

In [51]:
import gc

In [52]:
try:
    del lsa_values
    del top_images_by_lsa
    del top_labels_by_lsa
    del image_sets_lsa
    del label_sets_lsa
    del models_lsa
except:
    print("Error")

In [53]:
gc.collect()

316972

## Training guided by DSA

In [54]:
dsa_direction = "D:/guided-retraining/data/"+dataset+"/dsa_values.npy"

In [55]:
#dsa_direction = "C:/Users/fjdurlop/Documents/upc/upc-july/data/"+dataset+"/dsa_values.npy"

In [56]:
dsa_values = np.load(dsa_direction)[len(x_train):] 
print(len(dsa_values))

6999


In [57]:
top_images_by_dsa = utils.get_x_of_indexes(list(np.flip(np.argsort(dsa_values))),x_adversary_training)
top_labels_by_dsa = utils.get_x_of_indexes(list(np.flip(np.argsort(dsa_values))),y_adversary_training)

In [58]:
m = n_data_points
n = 0
image_sets_dsa = []
label_sets_dsa = []


for i in range((len(top_images_by_dsa)//m)):
    print(i,":")
    if (i+1 >= ((len(top_images_by_dsa)//m))):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_dsa)%m))
        top_images_by_dsa_n = np.array(top_images_by_dsa[:n+m+(len(top_images_by_dsa)%m)])
        top_labels_by_dsa_n = np.array(top_labels_by_dsa[:n+m+(len(top_images_by_dsa)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_dsa_n = np.array(top_images_by_dsa[:n+m])
        top_labels_by_dsa_n = np.array(top_labels_by_dsa[:n+m])
    image_sets_dsa.append(top_images_by_dsa_n)
    label_sets_dsa.append(top_labels_by_dsa_n)
    print(len(top_images_by_dsa_n))
    n += m



0 :
0  ->  340
340
1 :
0  ->  680
680
2 :
0  ->  1020
1020
3 :
0  ->  1360
1360
4 :
0  ->  1700
1700
5 :
0  ->  2040
2040
6 :
0  ->  2380
2380
7 :
0  ->  2720
2720
8 :
0  ->  3060
3060
9 :
0  ->  3400
3400
10 :
0  ->  3740
3740
11 :
0  ->  4080
4080
12 :
0  ->  4420
4420
13 :
0  ->  4760
4760
14 :
0  ->  5100
5100
15 :
0  ->  5440
5440
16 :
0  ->  5780
5780
17 :
0  ->  6120
6120
18 :
0  ->  6460
6460
19 :
Last
0  ->  6999
6999


In [59]:
print(model_dir)

models_dsa = []
for i in range(len(label_sets_dsa)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_dsa.append(model)


D:/guided-retraining/models/model_fashion_2
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [60]:
n=0
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

0
Epoch 1/10
6/6 [==============================] - 7s 1s/step - loss: 1.3093 - accuracy: 0.5029 - val_loss: 0.2801 - val_accuracy: 0.9015
Epoch 2/10
6/6 [==============================] - 5s 1s/step - loss: 0.9539 - accuracy: 0.6324 - val_loss: 0.3072 - val_accuracy: 0.8899
Epoch 3/10
6/6 [==============================] - 5s 1s/step - loss: 0.7443 - accuracy: 0.7206 - val_loss: 0.3112 - val_accuracy: 0.8891
Epoch 4/10
6/6 [==============================] - 5s 1s/step - loss: 0.7412 - accuracy: 0.7118 - val_loss: 0.3108 - val_accuracy: 0.8878
Epoch 5/10
6/6 [==============================] - 5s 1s/step - loss: 0.6309 - accuracy: 0.7412 - val_loss: 0.3177 - val_accuracy: 0.8874
Epoch 6/10
6/6 [==============================] - 5s 1s/step - loss: 0.5479 - accuracy: 0.8118 - val_loss: 0.3234 - val_accuracy: 0.8858
Epoch 7/10
6/6 [==============================] - 5s 1s/step - loss: 0.4976 - accuracy: 0.8265 - val_loss: 0.3269 - val_accuracy: 0.8856
Epoch 8/10
6/6 [=======================

In [61]:
n=1
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

1
Epoch 1/10
11/11 [==============================] - 8s 616ms/step - loss: 1.2424 - accuracy: 0.5500 - val_loss: 0.2998 - val_accuracy: 0.8935
Epoch 2/10
11/11 [==============================] - 6s 568ms/step - loss: 0.8973 - accuracy: 0.6765 - val_loss: 0.3239 - val_accuracy: 0.8865
Epoch 3/10
11/11 [==============================] - 6s 551ms/step - loss: 0.6935 - accuracy: 0.7250 - val_loss: 0.3293 - val_accuracy: 0.8838
Epoch 4/10
11/11 [==============================] - 6s 571ms/step - loss: 0.6518 - accuracy: 0.7529 - val_loss: 0.3234 - val_accuracy: 0.8860
Epoch 5/10
11/11 [==============================] - 6s 561ms/step - loss: 0.5975 - accuracy: 0.7544 - val_loss: 0.3254 - val_accuracy: 0.8841
Epoch 6/10
11/11 [==============================] - 6s 549ms/step - loss: 0.5465 - accuracy: 0.8015 - val_loss: 0.3261 - val_accuracy: 0.8844
Epoch 7/10
11/11 [==============================] - 6s 579ms/step - loss: 0.4843 - accuracy: 0.8191 - val_loss: 0.3158 - val_accuracy: 0.8886
Epoc

In [62]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

2
Epoch 1/10
16/16 [==============================] - 8s 420ms/step - loss: 1.1775 - accuracy: 0.5755 - val_loss: 0.2992 - val_accuracy: 0.8916
Epoch 2/10
16/16 [==============================] - 6s 405ms/step - loss: 0.8336 - accuracy: 0.6882 - val_loss: 0.3089 - val_accuracy: 0.8868
Epoch 3/10
16/16 [==============================] - 6s 393ms/step - loss: 0.6873 - accuracy: 0.7333 - val_loss: 0.3195 - val_accuracy: 0.8807
Epoch 4/10
16/16 [==============================] - 6s 403ms/step - loss: 0.6190 - accuracy: 0.7667 - val_loss: 0.3149 - val_accuracy: 0.8889
Epoch 5/10
16/16 [==============================] - 6s 409ms/step - loss: 0.5453 - accuracy: 0.7804 - val_loss: 0.3230 - val_accuracy: 0.8854
Epoch 6/10
16/16 [==============================] - 6s 411ms/step - loss: 0.5110 - accuracy: 0.8098 - val_loss: 0.3160 - val_accuracy: 0.8884
Epoch 7/10
16/16 [==============================] - 6s 404ms/step - loss: 0.4922 - accuracy: 0.8265 - val_loss: 0.3317 - val_accuracy: 0.8849
Epoc

In [63]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

3
Epoch 1/10
22/22 [==============================] - 8s 322ms/step - loss: 1.1221 - accuracy: 0.5875 - val_loss: 0.2999 - val_accuracy: 0.8862
Epoch 2/10
22/22 [==============================] - 7s 308ms/step - loss: 0.7995 - accuracy: 0.7103 - val_loss: 0.3142 - val_accuracy: 0.8793
Epoch 3/10
22/22 [==============================] - 6s 304ms/step - loss: 0.7477 - accuracy: 0.7140 - val_loss: 0.3269 - val_accuracy: 0.8759
Epoch 4/10
22/22 [==============================] - 7s 308ms/step - loss: 0.6392 - accuracy: 0.7551 - val_loss: 0.3178 - val_accuracy: 0.8842
Epoch 5/10
22/22 [==============================] - 6s 306ms/step - loss: 0.5877 - accuracy: 0.7853 - val_loss: 0.3228 - val_accuracy: 0.8822
Epoch 6/10
22/22 [==============================] - 7s 315ms/step - loss: 0.5485 - accuracy: 0.7890 - val_loss: 0.3338 - val_accuracy: 0.8825
Epoch 7/10
22/22 [==============================] - 7s 310ms/step - loss: 0.4977 - accuracy: 0.8103 - val_loss: 0.3201 - val_accuracy: 0.8871
Epoc

In [64]:
#
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

4
Epoch 1/10
27/27 [==============================] - 8s 276ms/step - loss: 1.1026 - accuracy: 0.5788 - val_loss: 0.3070 - val_accuracy: 0.8824
Epoch 2/10
27/27 [==============================] - 7s 261ms/step - loss: 0.8143 - accuracy: 0.6818 - val_loss: 0.3130 - val_accuracy: 0.8821
Epoch 3/10
27/27 [==============================] - 7s 261ms/step - loss: 0.7182 - accuracy: 0.7247 - val_loss: 0.3228 - val_accuracy: 0.8784
Epoch 4/10
27/27 [==============================] - 7s 264ms/step - loss: 0.6493 - accuracy: 0.7518 - val_loss: 0.3197 - val_accuracy: 0.8814
Epoch 5/10
27/27 [==============================] - 7s 262ms/step - loss: 0.6204 - accuracy: 0.7594 - val_loss: 0.3142 - val_accuracy: 0.8851
Epoch 6/10
27/27 [==============================] - 7s 259ms/step - loss: 0.6087 - accuracy: 0.7788 - val_loss: 0.3059 - val_accuracy: 0.8868
Epoch 7/10
27/27 [==============================] - 7s 263ms/step - loss: 0.5395 - accuracy: 0.7912 - val_loss: 0.3239 - val_accuracy: 0.8837
Epoc

In [65]:
n=5
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

5
Epoch 1/10
32/32 [==============================] - 9s 242ms/step - loss: 1.0801 - accuracy: 0.5897 - val_loss: 0.2996 - val_accuracy: 0.8863
Epoch 2/10
32/32 [==============================] - 7s 235ms/step - loss: 0.7905 - accuracy: 0.6985 - val_loss: 0.3048 - val_accuracy: 0.8836
Epoch 3/10
32/32 [==============================] - 8s 240ms/step - loss: 0.7129 - accuracy: 0.7230 - val_loss: 0.3088 - val_accuracy: 0.8787
Epoch 4/10
32/32 [==============================] - 7s 239ms/step - loss: 0.6374 - accuracy: 0.7569 - val_loss: 0.3070 - val_accuracy: 0.8811
Epoch 5/10
32/32 [==============================] - 8s 248ms/step - loss: 0.5993 - accuracy: 0.7691 - val_loss: 0.3123 - val_accuracy: 0.8839
Epoch 6/10
32/32 [==============================] - 7s 239ms/step - loss: 0.5681 - accuracy: 0.7887 - val_loss: 0.2974 - val_accuracy: 0.8886
Epoch 7/10
32/32 [==============================] - 7s 237ms/step - loss: 0.5523 - accuracy: 0.7985 - val_loss: 0.3051 - val_accuracy: 0.8878
Epoc

In [66]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

6
Epoch 1/10
38/38 [==============================] - 9s 218ms/step - loss: 1.0777 - accuracy: 0.6034 - val_loss: 0.3024 - val_accuracy: 0.8792
Epoch 2/10
38/38 [==============================] - 8s 217ms/step - loss: 0.8490 - accuracy: 0.6845 - val_loss: 0.2870 - val_accuracy: 0.8884
Epoch 3/10
38/38 [==============================] - 8s 220ms/step - loss: 0.7296 - accuracy: 0.7071 - val_loss: 0.3111 - val_accuracy: 0.8799
Epoch 4/10
38/38 [==============================] - 8s 219ms/step - loss: 0.6721 - accuracy: 0.7479 - val_loss: 0.2971 - val_accuracy: 0.8845
Epoch 5/10
38/38 [==============================] - 8s 208ms/step - loss: 0.6159 - accuracy: 0.7592 - val_loss: 0.3056 - val_accuracy: 0.8872
Epoch 6/10
38/38 [==============================] - 8s 204ms/step - loss: 0.5902 - accuracy: 0.7735 - val_loss: 0.2973 - val_accuracy: 0.8934
Epoch 7/10
38/38 [==============================] - 8s 209ms/step - loss: 0.5200 - accuracy: 0.8042 - val_loss: 0.3077 - val_accuracy: 0.8864
Epoc

In [67]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

7
Epoch 1/10
43/43 [==============================] - 10s 208ms/step - loss: 1.0572 - accuracy: 0.6154 - val_loss: 0.3098 - val_accuracy: 0.8789
Epoch 2/10
43/43 [==============================] - 8s 195ms/step - loss: 0.7920 - accuracy: 0.7007 - val_loss: 0.3039 - val_accuracy: 0.8779
Epoch 3/10
43/43 [==============================] - 8s 195ms/step - loss: 0.6817 - accuracy: 0.7357 - val_loss: 0.2976 - val_accuracy: 0.8831
Epoch 4/10
43/43 [==============================] - 9s 201ms/step - loss: 0.6440 - accuracy: 0.7518 - val_loss: 0.2981 - val_accuracy: 0.8851
Epoch 5/10
43/43 [==============================] - 8s 194ms/step - loss: 0.5928 - accuracy: 0.7846 - val_loss: 0.3061 - val_accuracy: 0.8846
Epoch 6/10
43/43 [==============================] - 8s 195ms/step - loss: 0.5452 - accuracy: 0.8022 - val_loss: 0.3049 - val_accuracy: 0.8871
Epoch 7/10
43/43 [==============================] - 8s 194ms/step - loss: 0.5292 - accuracy: 0.8059 - val_loss: 0.3034 - val_accuracy: 0.8854
Epo

In [68]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

8
Epoch 1/10
48/48 [==============================] - 10s 190ms/step - loss: 0.9782 - accuracy: 0.6366 - val_loss: 0.3037 - val_accuracy: 0.8840
Epoch 2/10
48/48 [==============================] - 9s 184ms/step - loss: 0.7565 - accuracy: 0.7098 - val_loss: 0.3086 - val_accuracy: 0.8769
Epoch 3/10
48/48 [==============================] - 9s 187ms/step - loss: 0.6797 - accuracy: 0.7373 - val_loss: 0.3056 - val_accuracy: 0.8786
Epoch 4/10
48/48 [==============================] - 9s 181ms/step - loss: 0.6177 - accuracy: 0.7657 - val_loss: 0.2985 - val_accuracy: 0.8861
Epoch 5/10
48/48 [==============================] - 9s 193ms/step - loss: 0.5724 - accuracy: 0.7886 - val_loss: 0.2974 - val_accuracy: 0.8887
Epoch 6/10
48/48 [==============================] - 9s 185ms/step - loss: 0.5312 - accuracy: 0.8065 - val_loss: 0.2996 - val_accuracy: 0.8880
Epoch 7/10
48/48 [==============================] - 9s 186ms/step - loss: 0.4775 - accuracy: 0.8173 - val_loss: 0.2976 - val_accuracy: 0.8927
Epo

In [69]:
#
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

9
Epoch 1/10
54/54 [==============================] - 11s 176ms/step - loss: 0.8958 - accuracy: 0.6603 - val_loss: 0.3025 - val_accuracy: 0.8824
Epoch 2/10
54/54 [==============================] - 9s 170ms/step - loss: 0.7376 - accuracy: 0.7250 - val_loss: 0.3182 - val_accuracy: 0.8801
Epoch 3/10
54/54 [==============================] - 9s 172ms/step - loss: 0.6310 - accuracy: 0.7559 - val_loss: 0.3124 - val_accuracy: 0.8832
Epoch 4/10
54/54 [==============================] - 9s 171ms/step - loss: 0.5928 - accuracy: 0.7788 - val_loss: 0.2991 - val_accuracy: 0.8841
Epoch 5/10
54/54 [==============================] - 9s 170ms/step - loss: 0.5537 - accuracy: 0.8003 - val_loss: 0.3121 - val_accuracy: 0.8784
Epoch 6/10
54/54 [==============================] - 9s 170ms/step - loss: 0.5173 - accuracy: 0.8097 - val_loss: 0.2986 - val_accuracy: 0.8894
Epoch 7/10
54/54 [==============================] - 9s 170ms/step - loss: 0.4774 - accuracy: 0.8309 - val_loss: 0.3053 - val_accuracy: 0.8864
Epo

In [70]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

10
Epoch 1/10
59/59 [==============================] - 12s 172ms/step - loss: 0.8447 - accuracy: 0.6909 - val_loss: 0.2939 - val_accuracy: 0.8823
Epoch 2/10
59/59 [==============================] - 10s 163ms/step - loss: 0.6331 - accuracy: 0.7618 - val_loss: 0.3071 - val_accuracy: 0.8794
Epoch 3/10
59/59 [==============================] - 10s 165ms/step - loss: 0.5721 - accuracy: 0.7933 - val_loss: 0.2984 - val_accuracy: 0.8844
Epoch 4/10
59/59 [==============================] - 10s 164ms/step - loss: 0.5419 - accuracy: 0.8043 - val_loss: 0.2931 - val_accuracy: 0.8909
Epoch 5/10
59/59 [==============================] - 10s 165ms/step - loss: 0.4832 - accuracy: 0.8222 - val_loss: 0.2983 - val_accuracy: 0.8903
Epoch 6/10
59/59 [==============================] - 10s 165ms/step - loss: 0.4433 - accuracy: 0.8396 - val_loss: 0.3116 - val_accuracy: 0.8867
Epoch 7/10
59/59 [==============================] - 10s 164ms/step - loss: 0.4214 - accuracy: 0.8457 - val_loss: 0.3123 - val_accuracy: 0.8

In [71]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

11
Epoch 1/10
64/64 [==============================] - 11s 160ms/step - loss: 0.7888 - accuracy: 0.7100 - val_loss: 0.2953 - val_accuracy: 0.8865
Epoch 2/10
64/64 [==============================] - 10s 157ms/step - loss: 0.6106 - accuracy: 0.7779 - val_loss: 0.3030 - val_accuracy: 0.8848
Epoch 3/10
64/64 [==============================] - 10s 158ms/step - loss: 0.5326 - accuracy: 0.8017 - val_loss: 0.2855 - val_accuracy: 0.8918
Epoch 4/10
64/64 [==============================] - 10s 163ms/step - loss: 0.5006 - accuracy: 0.8167 - val_loss: 0.2930 - val_accuracy: 0.8875
Epoch 5/10
64/64 [==============================] - 10s 160ms/step - loss: 0.4498 - accuracy: 0.8333 - val_loss: 0.2923 - val_accuracy: 0.8889
Epoch 6/10
64/64 [==============================] - 10s 157ms/step - loss: 0.4160 - accuracy: 0.8517 - val_loss: 0.3031 - val_accuracy: 0.8859
Epoch 7/10
64/64 [==============================] - 10s 158ms/step - loss: 0.4067 - accuracy: 0.8544 - val_loss: 0.2883 - val_accuracy: 0.8

In [72]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

12
Epoch 1/10
70/70 [==============================] - 13s 159ms/step - loss: 0.7578 - accuracy: 0.7231 - val_loss: 0.2942 - val_accuracy: 0.8859
Epoch 2/10
70/70 [==============================] - 10s 140ms/step - loss: 0.5746 - accuracy: 0.7835 - val_loss: 0.3000 - val_accuracy: 0.8864
Epoch 3/10
70/70 [==============================] - 10s 149ms/step - loss: 0.5038 - accuracy: 0.8090 - val_loss: 0.2943 - val_accuracy: 0.8891
Epoch 4/10
70/70 [==============================] - 11s 154ms/step - loss: 0.4872 - accuracy: 0.8210 - val_loss: 0.3074 - val_accuracy: 0.8819
Epoch 5/10
70/70 [==============================] - 11s 158ms/step - loss: 0.4455 - accuracy: 0.8391 - val_loss: 0.2968 - val_accuracy: 0.8867
Epoch 6/10
70/70 [==============================] - 11s 154ms/step - loss: 0.4154 - accuracy: 0.8534 - val_loss: 0.3138 - val_accuracy: 0.8863
Epoch 7/10
70/70 [==============================] - 10s 149ms/step - loss: 0.3817 - accuracy: 0.8627 - val_loss: 0.2997 - val_accuracy: 0.8

In [73]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

13
Epoch 1/10
75/75 [==============================] - 13s 152ms/step - loss: 0.7226 - accuracy: 0.7284 - val_loss: 0.2883 - val_accuracy: 0.8889
Epoch 2/10
75/75 [==============================] - 11s 149ms/step - loss: 0.5574 - accuracy: 0.7870 - val_loss: 0.2927 - val_accuracy: 0.8901
Epoch 3/10
75/75 [==============================] - 11s 149ms/step - loss: 0.5221 - accuracy: 0.8080 - val_loss: 0.2891 - val_accuracy: 0.8926
Epoch 4/10
75/75 [==============================] - 11s 148ms/step - loss: 0.4765 - accuracy: 0.8242 - val_loss: 0.2905 - val_accuracy: 0.8917
Epoch 5/10
75/75 [==============================] - 11s 148ms/step - loss: 0.4549 - accuracy: 0.8359 - val_loss: 0.2872 - val_accuracy: 0.8939
Epoch 6/10
75/75 [==============================] - 11s 148ms/step - loss: 0.4026 - accuracy: 0.8529 - val_loss: 0.2905 - val_accuracy: 0.8935
Epoch 7/10
75/75 [==============================] - 11s 149ms/step - loss: 0.3742 - accuracy: 0.8660 - val_loss: 0.2854 - val_accuracy: 0.8

In [74]:
#
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

14
Epoch 1/10
80/80 [==============================] - 13s 148ms/step - loss: 0.6948 - accuracy: 0.7425 - val_loss: 0.2884 - val_accuracy: 0.8888
Epoch 2/10
80/80 [==============================] - 12s 145ms/step - loss: 0.5571 - accuracy: 0.7878 - val_loss: 0.2819 - val_accuracy: 0.8967
Epoch 3/10
80/80 [==============================] - 12s 146ms/step - loss: 0.5124 - accuracy: 0.8118 - val_loss: 0.2802 - val_accuracy: 0.8908
Epoch 4/10
80/80 [==============================] - 12s 152ms/step - loss: 0.4788 - accuracy: 0.8255 - val_loss: 0.2853 - val_accuracy: 0.8921
Epoch 5/10
80/80 [==============================] - 12s 146ms/step - loss: 0.4487 - accuracy: 0.8376 - val_loss: 0.2950 - val_accuracy: 0.8891
Epoch 6/10
80/80 [==============================] - 11s 144ms/step - loss: 0.4199 - accuracy: 0.8473 - val_loss: 0.2878 - val_accuracy: 0.8980
Epoch 7/10
80/80 [==============================] - 11s 143ms/step - loss: 0.3866 - accuracy: 0.8582 - val_loss: 0.2837 - val_accuracy: 0.8

In [75]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

15
Epoch 1/10
85/85 [==============================] - 13s 142ms/step - loss: 0.6996 - accuracy: 0.7410 - val_loss: 0.2828 - val_accuracy: 0.8976
Epoch 2/10
85/85 [==============================] - 12s 139ms/step - loss: 0.5666 - accuracy: 0.7925 - val_loss: 0.2940 - val_accuracy: 0.8906
Epoch 3/10
85/85 [==============================] - 12s 142ms/step - loss: 0.5064 - accuracy: 0.8197 - val_loss: 0.2800 - val_accuracy: 0.8966
Epoch 4/10
85/85 [==============================] - 12s 137ms/step - loss: 0.4719 - accuracy: 0.8228 - val_loss: 0.2856 - val_accuracy: 0.8952
Epoch 5/10
85/85 [==============================] - 12s 138ms/step - loss: 0.4428 - accuracy: 0.8450 - val_loss: 0.2855 - val_accuracy: 0.8966
Epoch 6/10
85/85 [==============================] - 12s 138ms/step - loss: 0.4073 - accuracy: 0.8577 - val_loss: 0.2882 - val_accuracy: 0.8971
Epoch 7/10
85/85 [==============================] - 11s 135ms/step - loss: 0.3805 - accuracy: 0.8687 - val_loss: 0.2850 - val_accuracy: 0.9

In [76]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

16
Epoch 1/10
91/91 [==============================] - 14s 142ms/step - loss: 0.6468 - accuracy: 0.7611 - val_loss: 0.2915 - val_accuracy: 0.8891
Epoch 2/10
91/91 [==============================] - 13s 141ms/step - loss: 0.5267 - accuracy: 0.8087 - val_loss: 0.2786 - val_accuracy: 0.8967
Epoch 3/10
91/91 [==============================] - 13s 144ms/step - loss: 0.4854 - accuracy: 0.8240 - val_loss: 0.2833 - val_accuracy: 0.8978
Epoch 4/10
91/91 [==============================] - 13s 138ms/step - loss: 0.4346 - accuracy: 0.8388 - val_loss: 0.2943 - val_accuracy: 0.8924
Epoch 5/10
91/91 [==============================] - 13s 139ms/step - loss: 0.4236 - accuracy: 0.8476 - val_loss: 0.2746 - val_accuracy: 0.9003
Epoch 6/10
91/91 [==============================] - 13s 138ms/step - loss: 0.3934 - accuracy: 0.8600 - val_loss: 0.2790 - val_accuracy: 0.9032
Epoch 7/10
91/91 [==============================] - 12s 137ms/step - loss: 0.3665 - accuracy: 0.8706 - val_loss: 0.2836 - val_accuracy: 0.8

In [77]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

17
Epoch 1/10
96/96 [==============================] - 15s 138ms/step - loss: 0.6256 - accuracy: 0.7750 - val_loss: 0.2796 - val_accuracy: 0.8916
Epoch 2/10
96/96 [==============================] - 13s 136ms/step - loss: 0.5164 - accuracy: 0.8165 - val_loss: 0.2767 - val_accuracy: 0.8949
Epoch 3/10
96/96 [==============================] - 13s 135ms/step - loss: 0.4608 - accuracy: 0.8302 - val_loss: 0.2783 - val_accuracy: 0.8931
Epoch 4/10
96/96 [==============================] - 13s 135ms/step - loss: 0.4458 - accuracy: 0.8415 - val_loss: 0.2881 - val_accuracy: 0.8919
Epoch 5/10
96/96 [==============================] - 13s 134ms/step - loss: 0.4079 - accuracy: 0.8547 - val_loss: 0.2779 - val_accuracy: 0.8984
Epoch 6/10
96/96 [==============================] - 13s 134ms/step - loss: 0.3709 - accuracy: 0.8704 - val_loss: 0.2856 - val_accuracy: 0.8996
Epoch 7/10
96/96 [==============================] - 13s 134ms/step - loss: 0.3530 - accuracy: 0.8747 - val_loss: 0.2796 - val_accuracy: 0.9

In [78]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

18
Epoch 1/10
101/101 [==============================] - 15s 134ms/step - loss: 0.5901 - accuracy: 0.7921 - val_loss: 0.2844 - val_accuracy: 0.8863
Epoch 2/10
101/101 [==============================] - 13s 131ms/step - loss: 0.4876 - accuracy: 0.8310 - val_loss: 0.2848 - val_accuracy: 0.8936
Epoch 3/10
101/101 [==============================] - 13s 130ms/step - loss: 0.4499 - accuracy: 0.8373 - val_loss: 0.2775 - val_accuracy: 0.8980
Epoch 4/10
101/101 [==============================] - 13s 130ms/step - loss: 0.4161 - accuracy: 0.8599 - val_loss: 0.2760 - val_accuracy: 0.8991
Epoch 5/10
101/101 [==============================] - 13s 132ms/step - loss: 0.3948 - accuracy: 0.8611 - val_loss: 0.2794 - val_accuracy: 0.8976
Epoch 6/10
101/101 [==============================] - 13s 132ms/step - loss: 0.3697 - accuracy: 0.8697 - val_loss: 0.2863 - val_accuracy: 0.8992
Epoch 7/10
101/101 [==============================] - 13s 131ms/step - loss: 0.3391 - accuracy: 0.8824 - val_loss: 0.2794 - val

In [79]:
#
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

19
Epoch 1/10
110/110 [==============================] - 16s 133ms/step - loss: 0.5634 - accuracy: 0.8028 - val_loss: 0.2922 - val_accuracy: 0.8866
Epoch 2/10
110/110 [==============================] - 14s 130ms/step - loss: 0.4586 - accuracy: 0.8351 - val_loss: 0.2820 - val_accuracy: 0.8942
Epoch 3/10
110/110 [==============================] - 14s 125ms/step - loss: 0.4066 - accuracy: 0.8574 - val_loss: 0.2812 - val_accuracy: 0.8966
Epoch 4/10
110/110 [==============================] - 14s 129ms/step - loss: 0.3762 - accuracy: 0.8627 - val_loss: 0.2844 - val_accuracy: 0.8958
Epoch 5/10
110/110 [==============================] - 15s 135ms/step - loss: 0.3490 - accuracy: 0.8710 - val_loss: 0.2830 - val_accuracy: 0.9016
Epoch 6/10
110/110 [==============================] - 14s 131ms/step - loss: 0.3316 - accuracy: 0.8874 - val_loss: 0.3105 - val_accuracy: 0.8941
Epoch 7/10
110/110 [==============================] - 14s 132ms/step - loss: 0.3024 - accuracy: 0.8958 - val_loss: 0.2981 - val

In [80]:
#new_model_dsa_dir  = "C:/Users/fjdurlop/Documents/upc/models/"+dataset+"/C3/"+dataset+"_model_c3_aug_dsa_e1"
new_model_dsa_dir  = "D:/models/aug_22/"+dataset+"/C3/"+dataset+"_model_c3_may_dsa_e1"


i=0

for model in models_dsa:
    model.save(new_model_dsa_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_may_dsa_e1_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_may_dsa_e1_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_may_dsa_e1_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_may_dsa_e1_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_may_dsa_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_may_dsa_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_may_dsa_e1_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_may_dsa_e1_7\assets
Model has been saved
INFO:tensorflow:Assets written t

In [81]:
try:
    del dsa_values
    del top_images_by_dsa
    del top_labels_by_dsa
    del image_sets_dsa
    del label_sets_dsa
    del models_dsa
except:
    print("Error")

In [82]:
import gc
gc.collect()

327980

## Uncertainty

In [19]:
#dg_direction = "C:/Users/fjdurlop/Documents/upc/upc-july/data/"+dataset+"/deep_gini_values.npy"

#deep_gini_values = np.load(dg_direction)
dg_direction = "D:/guided-retraining/data/"+dataset+"/deep_gini_values.npy"

deep_gini_values = np.load(dg_direction)[len(x_train):] 
print(len(deep_gini_values))

6999


In [20]:
# Obtaining top n images by dg values
top_images_by_dg  = utils.get_x_of_indexes(list(np.flip(np.argsort(deep_gini_values))),x_adversary_training)
top_labels_by_dg = utils.get_x_of_indexes(list(np.flip(np.argsort(deep_gini_values))),y_adversary_training)

In [21]:
m = n_data_points
n = 0
image_sets_dg = []
label_sets_dg = []

# last
#for i in range(0,len(top_images_by_lsa)//m):

for i in range((len(top_images_by_dg)//m)):
    print(i,":")
    if (i+1 >= ((len(top_images_by_dg)//m))):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_dg)%m))
        top_images_by_dg_n = np.array(top_images_by_dg[:n+m+(len(top_images_by_dg)%m)])
        top_labels_by_dg_n = np.array(top_labels_by_dg[:n+m+(len(top_images_by_dg)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_dg_n = np.array(top_images_by_dg[:n+m])
        top_labels_by_dg_n = np.array(top_labels_by_dg[:n+m])
    image_sets_dg.append(top_images_by_dg_n)
    label_sets_dg.append(top_labels_by_dg_n)
    print(len(top_images_by_dg_n))
    n += m



0 :
0  ->  340
340
1 :
0  ->  680
680
2 :
0  ->  1020
1020
3 :
0  ->  1360
1360
4 :
0  ->  1700
1700
5 :
0  ->  2040
2040
6 :
0  ->  2380
2380
7 :
0  ->  2720
2720
8 :
0  ->  3060
3060
9 :
0  ->  3400
3400
10 :
0  ->  3740
3740
11 :
0  ->  4080
4080
12 :
0  ->  4420
4420
13 :
0  ->  4760
4760
14 :
0  ->  5100
5100
15 :
0  ->  5440
5440
16 :
0  ->  5780
5780
17 :
0  ->  6120
6120
18 :
0  ->  6460
6460
19 :
Last
0  ->  6999
6999


In [22]:
print(model_dir)
models_dg = []
#for i in range(len(label_sets_lsa)):
for i in range(len(label_sets_dg)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_dg.append(model)

D:/guided-retraining/models/model_fashion_2
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [87]:
n=0
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

0
Epoch 1/10
6/6 [==============================] - 7s 1s/step - loss: 1.5148 - accuracy: 0.4765 - val_loss: 0.2901 - val_accuracy: 0.8979
Epoch 2/10
6/6 [==============================] - 5s 1s/step - loss: 1.1188 - accuracy: 0.6000 - val_loss: 0.3109 - val_accuracy: 0.8911
Epoch 3/10
6/6 [==============================] - 5s 1s/step - loss: 0.9627 - accuracy: 0.6618 - val_loss: 0.3365 - val_accuracy: 0.8895
Epoch 4/10
6/6 [==============================] - 5s 1s/step - loss: 0.7582 - accuracy: 0.7265 - val_loss: 0.3590 - val_accuracy: 0.8847
Epoch 5/10
6/6 [==============================] - 5s 1s/step - loss: 0.7518 - accuracy: 0.7265 - val_loss: 0.3643 - val_accuracy: 0.8849
Epoch 6/10
6/6 [==============================] - 5s 1s/step - loss: 0.6995 - accuracy: 0.7412 - val_loss: 0.3715 - val_accuracy: 0.8846
Epoch 7/10
6/6 [==============================] - 5s 1s/step - loss: 0.7021 - accuracy: 0.7647 - val_loss: 0.3785 - val_accuracy: 0.8822
Epoch 8/10
6/6 [=======================

In [88]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

1
Epoch 1/10
11/11 [==============================] - 7s 592ms/step - loss: 1.2421 - accuracy: 0.5265 - val_loss: 0.2852 - val_accuracy: 0.8985
Epoch 2/10
11/11 [==============================] - 6s 571ms/step - loss: 0.9577 - accuracy: 0.6412 - val_loss: 0.2797 - val_accuracy: 0.9011
Epoch 3/10
11/11 [==============================] - 6s 562ms/step - loss: 0.7538 - accuracy: 0.7088 - val_loss: 0.2904 - val_accuracy: 0.8991
Epoch 4/10
11/11 [==============================] - 6s 588ms/step - loss: 0.7710 - accuracy: 0.6971 - val_loss: 0.3036 - val_accuracy: 0.8959
Epoch 5/10
11/11 [==============================] - 6s 568ms/step - loss: 0.7306 - accuracy: 0.7250 - val_loss: 0.3082 - val_accuracy: 0.8967
Epoch 6/10
11/11 [==============================] - 6s 572ms/step - loss: 0.6534 - accuracy: 0.7529 - val_loss: 0.3147 - val_accuracy: 0.8982
Epoch 7/10
11/11 [==============================] - 6s 574ms/step - loss: 0.5952 - accuracy: 0.7662 - val_loss: 0.3244 - val_accuracy: 0.8968
Epoc

In [89]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

2
Epoch 1/10
16/16 [==============================] - 8s 443ms/step - loss: 1.2271 - accuracy: 0.5206 - val_loss: 0.2724 - val_accuracy: 0.9009
Epoch 2/10
16/16 [==============================] - 6s 427ms/step - loss: 0.9658 - accuracy: 0.5971 - val_loss: 0.2697 - val_accuracy: 0.9034
Epoch 3/10
16/16 [==============================] - 6s 428ms/step - loss: 0.8629 - accuracy: 0.6657 - val_loss: 0.2858 - val_accuracy: 0.9014
Epoch 4/10
16/16 [==============================] - 6s 416ms/step - loss: 0.7639 - accuracy: 0.7069 - val_loss: 0.2818 - val_accuracy: 0.9017
Epoch 5/10
16/16 [==============================] - 6s 414ms/step - loss: 0.7176 - accuracy: 0.7373 - val_loss: 0.2966 - val_accuracy: 0.8991
Epoch 6/10
16/16 [==============================] - 6s 413ms/step - loss: 0.6892 - accuracy: 0.7353 - val_loss: 0.3044 - val_accuracy: 0.9015
Epoch 7/10
16/16 [==============================] - 7s 432ms/step - loss: 0.6660 - accuracy: 0.7382 - val_loss: 0.3071 - val_accuracy: 0.8989
Epoc

In [90]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

3
Epoch 1/10
22/22 [==============================] - 8s 314ms/step - loss: 1.1574 - accuracy: 0.5287 - val_loss: 0.2704 - val_accuracy: 0.8995
Epoch 2/10
22/22 [==============================] - 6s 304ms/step - loss: 0.9457 - accuracy: 0.6074 - val_loss: 0.2872 - val_accuracy: 0.8956
Epoch 3/10
22/22 [==============================] - 6s 303ms/step - loss: 0.8114 - accuracy: 0.6779 - val_loss: 0.2851 - val_accuracy: 0.8992
Epoch 4/10
22/22 [==============================] - 6s 305ms/step - loss: 0.8276 - accuracy: 0.6669 - val_loss: 0.2852 - val_accuracy: 0.9019
Epoch 5/10
22/22 [==============================] - 7s 316ms/step - loss: 0.7320 - accuracy: 0.7096 - val_loss: 0.2879 - val_accuracy: 0.8971
Epoch 6/10
22/22 [==============================] - 7s 314ms/step - loss: 0.7238 - accuracy: 0.7118 - val_loss: 0.2913 - val_accuracy: 0.8990
Epoch 7/10
22/22 [==============================] - 6s 306ms/step - loss: 0.6771 - accuracy: 0.7213 - val_loss: 0.2894 - val_accuracy: 0.9014
Epoc

In [91]:
#
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

4
Epoch 1/10
27/27 [==============================] - 8s 268ms/step - loss: 1.1232 - accuracy: 0.5335 - val_loss: 0.2633 - val_accuracy: 0.9049
Epoch 2/10
27/27 [==============================] - 7s 255ms/step - loss: 0.8881 - accuracy: 0.6612 - val_loss: 0.2739 - val_accuracy: 0.9035
Epoch 3/10
27/27 [==============================] - 7s 259ms/step - loss: 0.8052 - accuracy: 0.6700 - val_loss: 0.2763 - val_accuracy: 0.8975
Epoch 4/10
27/27 [==============================] - 7s 259ms/step - loss: 0.7419 - accuracy: 0.7135 - val_loss: 0.2821 - val_accuracy: 0.9000
Epoch 5/10
27/27 [==============================] - 7s 279ms/step - loss: 0.7179 - accuracy: 0.7271 - val_loss: 0.2819 - val_accuracy: 0.8999
Epoch 6/10
27/27 [==============================] - 7s 261ms/step - loss: 0.6490 - accuracy: 0.7429 - val_loss: 0.2889 - val_accuracy: 0.9021
Epoch 7/10
27/27 [==============================] - 7s 271ms/step - loss: 0.6370 - accuracy: 0.7559 - val_loss: 0.2935 - val_accuracy: 0.9007
Epoc

In [92]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

5
Epoch 1/10
32/32 [==============================] - 9s 243ms/step - loss: 1.0778 - accuracy: 0.5598 - val_loss: 0.2824 - val_accuracy: 0.8985
Epoch 2/10
32/32 [==============================] - 7s 231ms/step - loss: 0.8734 - accuracy: 0.6480 - val_loss: 0.2729 - val_accuracy: 0.9010
Epoch 3/10
32/32 [==============================] - 7s 236ms/step - loss: 0.7916 - accuracy: 0.6730 - val_loss: 0.2703 - val_accuracy: 0.9035
Epoch 4/10
32/32 [==============================] - 7s 233ms/step - loss: 0.7133 - accuracy: 0.7137 - val_loss: 0.2778 - val_accuracy: 0.9027
Epoch 5/10
32/32 [==============================] - 7s 234ms/step - loss: 0.6972 - accuracy: 0.7250 - val_loss: 0.2779 - val_accuracy: 0.9010
Epoch 6/10
32/32 [==============================] - 7s 233ms/step - loss: 0.6676 - accuracy: 0.7417 - val_loss: 0.2733 - val_accuracy: 0.9052
Epoch 7/10
32/32 [==============================] - 7s 225ms/step - loss: 0.6442 - accuracy: 0.7529 - val_loss: 0.2841 - val_accuracy: 0.9046
Epoc

In [93]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

6
Epoch 1/10
38/38 [==============================] - 9s 198ms/step - loss: 1.0603 - accuracy: 0.5824 - val_loss: 0.2710 - val_accuracy: 0.8974
Epoch 2/10
38/38 [==============================] - 7s 192ms/step - loss: 0.8432 - accuracy: 0.6609 - val_loss: 0.2778 - val_accuracy: 0.8921
Epoch 3/10
38/38 [==============================] - 7s 194ms/step - loss: 0.7575 - accuracy: 0.7025 - val_loss: 0.2719 - val_accuracy: 0.9001
Epoch 4/10
38/38 [==============================] - 7s 198ms/step - loss: 0.6909 - accuracy: 0.7345 - val_loss: 0.2731 - val_accuracy: 0.9029
Epoch 5/10
38/38 [==============================] - 8s 209ms/step - loss: 0.6863 - accuracy: 0.7382 - val_loss: 0.2799 - val_accuracy: 0.8991
Epoch 6/10
38/38 [==============================] - 7s 200ms/step - loss: 0.6404 - accuracy: 0.7538 - val_loss: 0.2838 - val_accuracy: 0.9003
Epoch 7/10
38/38 [==============================] - 8s 202ms/step - loss: 0.6079 - accuracy: 0.7765 - val_loss: 0.2798 - val_accuracy: 0.9009
Epoc

In [94]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

7
Epoch 1/10
43/43 [==============================] - 10s 200ms/step - loss: 1.0148 - accuracy: 0.6070 - val_loss: 0.2650 - val_accuracy: 0.9013
Epoch 2/10
43/43 [==============================] - 8s 199ms/step - loss: 0.8347 - accuracy: 0.6842 - val_loss: 0.2753 - val_accuracy: 0.8965
Epoch 3/10
43/43 [==============================] - 8s 192ms/step - loss: 0.7583 - accuracy: 0.7022 - val_loss: 0.2737 - val_accuracy: 0.9002
Epoch 4/10
43/43 [==============================] - 8s 193ms/step - loss: 0.6978 - accuracy: 0.7294 - val_loss: 0.2896 - val_accuracy: 0.8909
Epoch 5/10
43/43 [==============================] - 8s 190ms/step - loss: 0.6721 - accuracy: 0.7449 - val_loss: 0.2786 - val_accuracy: 0.8991
Epoch 6/10
43/43 [==============================] - 8s 194ms/step - loss: 0.6384 - accuracy: 0.7596 - val_loss: 0.2818 - val_accuracy: 0.8988
Epoch 7/10
43/43 [==============================] - 8s 196ms/step - loss: 0.6056 - accuracy: 0.7662 - val_loss: 0.2860 - val_accuracy: 0.9000
Epo

In [95]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

8
Epoch 1/10
48/48 [==============================] - 10s 189ms/step - loss: 0.9966 - accuracy: 0.5980 - val_loss: 0.2746 - val_accuracy: 0.8939
Epoch 2/10
48/48 [==============================] - 9s 182ms/step - loss: 0.7913 - accuracy: 0.6967 - val_loss: 0.2719 - val_accuracy: 0.8947
Epoch 3/10
48/48 [==============================] - 9s 183ms/step - loss: 0.7523 - accuracy: 0.7069 - val_loss: 0.2699 - val_accuracy: 0.8979
Epoch 4/10
48/48 [==============================] - 9s 181ms/step - loss: 0.6992 - accuracy: 0.7386 - val_loss: 0.2706 - val_accuracy: 0.8979
Epoch 5/10
48/48 [==============================] - 9s 182ms/step - loss: 0.6681 - accuracy: 0.7497 - val_loss: 0.2702 - val_accuracy: 0.9029
Epoch 6/10
48/48 [==============================] - 9s 181ms/step - loss: 0.6173 - accuracy: 0.7667 - val_loss: 0.2850 - val_accuracy: 0.8996
Epoch 7/10
48/48 [==============================] - 9s 182ms/step - loss: 0.5986 - accuracy: 0.7699 - val_loss: 0.2755 - val_accuracy: 0.9017
Epo

In [96]:
#
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

9
Epoch 1/10
54/54 [==============================] - 11s 176ms/step - loss: 0.9698 - accuracy: 0.6335 - val_loss: 0.2779 - val_accuracy: 0.8962
Epoch 2/10
54/54 [==============================] - 9s 167ms/step - loss: 0.7743 - accuracy: 0.6982 - val_loss: 0.2714 - val_accuracy: 0.8953
Epoch 3/10
54/54 [==============================] - 9s 167ms/step - loss: 0.7059 - accuracy: 0.7235 - val_loss: 0.2861 - val_accuracy: 0.8941
Epoch 4/10
54/54 [==============================] - 9s 169ms/step - loss: 0.6781 - accuracy: 0.7424 - val_loss: 0.2846 - val_accuracy: 0.8939
Epoch 5/10
54/54 [==============================] - 9s 168ms/step - loss: 0.6375 - accuracy: 0.7615 - val_loss: 0.2745 - val_accuracy: 0.8994
Epoch 6/10
54/54 [==============================] - 9s 168ms/step - loss: 0.6052 - accuracy: 0.7779 - val_loss: 0.2868 - val_accuracy: 0.8982
Epoch 7/10
54/54 [==============================] - 9s 169ms/step - loss: 0.5840 - accuracy: 0.7844 - val_loss: 0.2921 - val_accuracy: 0.8998
Epo

In [97]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

10
Epoch 1/10
59/59 [==============================] - 11s 170ms/step - loss: 0.9441 - accuracy: 0.6455 - val_loss: 0.2751 - val_accuracy: 0.8953
Epoch 2/10
59/59 [==============================] - 10s 164ms/step - loss: 0.7690 - accuracy: 0.7115 - val_loss: 0.2698 - val_accuracy: 0.8957
Epoch 3/10
59/59 [==============================] - 10s 163ms/step - loss: 0.6982 - accuracy: 0.7302 - val_loss: 0.2732 - val_accuracy: 0.9015
Epoch 4/10
59/59 [==============================] - 10s 165ms/step - loss: 0.6653 - accuracy: 0.7455 - val_loss: 0.2808 - val_accuracy: 0.8956
Epoch 5/10
59/59 [==============================] - 10s 167ms/step - loss: 0.6121 - accuracy: 0.7698 - val_loss: 0.2701 - val_accuracy: 0.9014
Epoch 6/10
59/59 [==============================] - 10s 175ms/step - loss: 0.5810 - accuracy: 0.7743 - val_loss: 0.2809 - val_accuracy: 0.9017
Epoch 7/10
59/59 [==============================] - 10s 165ms/step - loss: 0.5339 - accuracy: 0.8059 - val_loss: 0.3074 - val_accuracy: 0.8

In [23]:
n=11
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

11
Epoch 1/10
64/64 [==============================] - 12s 156ms/step - loss: 0.9141 - accuracy: 0.6603 - val_loss: 0.2707 - val_accuracy: 0.8991
Epoch 2/10
64/64 [==============================] - 10s 150ms/step - loss: 0.7261 - accuracy: 0.7184 - val_loss: 0.2835 - val_accuracy: 0.8905
Epoch 3/10
64/64 [==============================] - 10s 151ms/step - loss: 0.6706 - accuracy: 0.7512 - val_loss: 0.2754 - val_accuracy: 0.8963
Epoch 4/10
64/64 [==============================] - 9s 147ms/step - loss: 0.6121 - accuracy: 0.7738 - val_loss: 0.2782 - val_accuracy: 0.8979
Epoch 5/10
64/64 [==============================] - 10s 150ms/step - loss: 0.5845 - accuracy: 0.7841 - val_loss: 0.2809 - val_accuracy: 0.8987
Epoch 6/10
64/64 [==============================] - 9s 149ms/step - loss: 0.5538 - accuracy: 0.8027 - val_loss: 0.2819 - val_accuracy: 0.9009
Epoch 7/10
64/64 [==============================] - 9s 149ms/step - loss: 0.5339 - accuracy: 0.8093 - val_loss: 0.2742 - val_accuracy: 0.9026

In [24]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

12
Epoch 1/10
70/70 [==============================] - 11s 146ms/step - loss: 0.8458 - accuracy: 0.6925 - val_loss: 0.2820 - val_accuracy: 0.8904
Epoch 2/10
70/70 [==============================] - 10s 144ms/step - loss: 0.7028 - accuracy: 0.7308 - val_loss: 0.2753 - val_accuracy: 0.8964
Epoch 3/10
70/70 [==============================] - 10s 139ms/step - loss: 0.6250 - accuracy: 0.7647 - val_loss: 0.2762 - val_accuracy: 0.8984
Epoch 4/10
70/70 [==============================] - 9s 136ms/step - loss: 0.5942 - accuracy: 0.7774 - val_loss: 0.2790 - val_accuracy: 0.8989
Epoch 5/10
70/70 [==============================] - 10s 142ms/step - loss: 0.5542 - accuracy: 0.7905 - val_loss: 0.2823 - val_accuracy: 0.8964
Epoch 6/10
70/70 [==============================] - 10s 139ms/step - loss: 0.5146 - accuracy: 0.8068 - val_loss: 0.2799 - val_accuracy: 0.8989
Epoch 7/10
70/70 [==============================] - 10s 140ms/step - loss: 0.4905 - accuracy: 0.8258 - val_loss: 0.2930 - val_accuracy: 0.89

In [25]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

13
Epoch 1/10
75/75 [==============================] - 12s 143ms/step - loss: 0.7979 - accuracy: 0.7149 - val_loss: 0.2673 - val_accuracy: 0.8965
Epoch 2/10
75/75 [==============================] - 10s 136ms/step - loss: 0.6525 - accuracy: 0.7601 - val_loss: 0.2768 - val_accuracy: 0.8954
Epoch 3/10
75/75 [==============================] - 10s 138ms/step - loss: 0.5868 - accuracy: 0.7866 - val_loss: 0.2868 - val_accuracy: 0.8908
Epoch 4/10
75/75 [==============================] - 10s 138ms/step - loss: 0.5522 - accuracy: 0.8019 - val_loss: 0.2786 - val_accuracy: 0.8982
Epoch 5/10
75/75 [==============================] - 10s 139ms/step - loss: 0.5219 - accuracy: 0.8122 - val_loss: 0.2840 - val_accuracy: 0.8991
Epoch 6/10
75/75 [==============================] - 10s 138ms/step - loss: 0.4886 - accuracy: 0.8225 - val_loss: 0.2902 - val_accuracy: 0.8957
Epoch 7/10
75/75 [==============================] - 10s 138ms/step - loss: 0.4674 - accuracy: 0.8376 - val_loss: 0.2928 - val_accuracy: 0.8

In [26]:
#
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

14
Epoch 1/10
80/80 [==============================] - 12s 136ms/step - loss: 0.7478 - accuracy: 0.7286 - val_loss: 0.2880 - val_accuracy: 0.8896
Epoch 2/10
80/80 [==============================] - 9s 115ms/step - loss: 0.6102 - accuracy: 0.7712 - val_loss: 0.2861 - val_accuracy: 0.8949
Epoch 3/10
80/80 [==============================] - 11s 135ms/step - loss: 0.5560 - accuracy: 0.8035 - val_loss: 0.2778 - val_accuracy: 0.8961
Epoch 4/10
80/80 [==============================] - 11s 134ms/step - loss: 0.5228 - accuracy: 0.8153 - val_loss: 0.2720 - val_accuracy: 0.9031
Epoch 5/10
80/80 [==============================] - 11s 134ms/step - loss: 0.4947 - accuracy: 0.8243 - val_loss: 0.2834 - val_accuracy: 0.8962
Epoch 6/10
80/80 [==============================] - 11s 134ms/step - loss: 0.4567 - accuracy: 0.8408 - val_loss: 0.2738 - val_accuracy: 0.8976
Epoch 7/10
80/80 [==============================] - 11s 134ms/step - loss: 0.4260 - accuracy: 0.8535 - val_loss: 0.2826 - val_accuracy: 0.89

In [27]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

15
Epoch 1/10
85/85 [==============================] - 13s 136ms/step - loss: 0.6948 - accuracy: 0.7498 - val_loss: 0.2846 - val_accuracy: 0.8942
Epoch 2/10
85/85 [==============================] - 11s 133ms/step - loss: 0.5669 - accuracy: 0.7917 - val_loss: 0.2893 - val_accuracy: 0.8948
Epoch 3/10
85/85 [==============================] - 11s 131ms/step - loss: 0.5224 - accuracy: 0.8165 - val_loss: 0.2808 - val_accuracy: 0.8970
Epoch 4/10
85/85 [==============================] - 11s 136ms/step - loss: 0.4836 - accuracy: 0.8206 - val_loss: 0.2721 - val_accuracy: 0.8994
Epoch 5/10
85/85 [==============================] - 11s 131ms/step - loss: 0.4552 - accuracy: 0.8382 - val_loss: 0.2811 - val_accuracy: 0.8981
Epoch 6/10
85/85 [==============================] - 11s 128ms/step - loss: 0.4222 - accuracy: 0.8528 - val_loss: 0.2886 - val_accuracy: 0.8964
Epoch 7/10
85/85 [==============================] - 11s 130ms/step - loss: 0.3879 - accuracy: 0.8625 - val_loss: 0.2858 - val_accuracy: 0.8

In [28]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

16
Epoch 1/10
91/91 [==============================] - 14s 134ms/step - loss: 0.6630 - accuracy: 0.7668 - val_loss: 0.2834 - val_accuracy: 0.8914
Epoch 2/10
91/91 [==============================] - 12s 129ms/step - loss: 0.5413 - accuracy: 0.8047 - val_loss: 0.2817 - val_accuracy: 0.8916
Epoch 3/10
91/91 [==============================] - 11s 126ms/step - loss: 0.4772 - accuracy: 0.8277 - val_loss: 0.2727 - val_accuracy: 0.8979
Epoch 4/10
91/91 [==============================] - 11s 126ms/step - loss: 0.4407 - accuracy: 0.8434 - val_loss: 0.2907 - val_accuracy: 0.8948
Epoch 5/10
91/91 [==============================] - 12s 130ms/step - loss: 0.4215 - accuracy: 0.8460 - val_loss: 0.2842 - val_accuracy: 0.8959
Epoch 6/10
91/91 [==============================] - 12s 127ms/step - loss: 0.4012 - accuracy: 0.8567 - val_loss: 0.2968 - val_accuracy: 0.8975
Epoch 7/10
91/91 [==============================] - 12s 131ms/step - loss: 0.3843 - accuracy: 0.8649 - val_loss: 0.2932 - val_accuracy: 0.8

In [29]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

17
Epoch 1/10
96/96 [==============================] - 14s 125ms/step - loss: 0.6474 - accuracy: 0.7632 - val_loss: 0.2878 - val_accuracy: 0.8894
Epoch 2/10
96/96 [==============================] - 11s 118ms/step - loss: 0.5166 - accuracy: 0.8126 - val_loss: 0.2774 - val_accuracy: 0.8980
Epoch 3/10
96/96 [==============================] - 11s 118ms/step - loss: 0.4661 - accuracy: 0.8324 - val_loss: 0.2798 - val_accuracy: 0.8971
Epoch 4/10
96/96 [==============================] - 12s 122ms/step - loss: 0.4287 - accuracy: 0.8412 - val_loss: 0.2897 - val_accuracy: 0.8923
Epoch 5/10
96/96 [==============================] - 12s 126ms/step - loss: 0.4202 - accuracy: 0.8511 - val_loss: 0.2832 - val_accuracy: 0.8963
Epoch 6/10
96/96 [==============================] - 12s 125ms/step - loss: 0.3790 - accuracy: 0.8645 - val_loss: 0.2851 - val_accuracy: 0.8996
Epoch 7/10
96/96 [==============================] - 12s 124ms/step - loss: 0.3618 - accuracy: 0.8745 - val_loss: 0.3055 - val_accuracy: 0.8

In [30]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

18
Epoch 1/10
101/101 [==============================] - 14s 127ms/step - loss: 0.6100 - accuracy: 0.7813 - val_loss: 0.2710 - val_accuracy: 0.8979
Epoch 2/10
101/101 [==============================] - 12s 122ms/step - loss: 0.4968 - accuracy: 0.8158 - val_loss: 0.2790 - val_accuracy: 0.8964
Epoch 3/10
101/101 [==============================] - 12s 124ms/step - loss: 0.4505 - accuracy: 0.8342 - val_loss: 0.2862 - val_accuracy: 0.8934
Epoch 4/10
101/101 [==============================] - 13s 125ms/step - loss: 0.4133 - accuracy: 0.8523 - val_loss: 0.2822 - val_accuracy: 0.8967
Epoch 5/10
101/101 [==============================] - 12s 120ms/step - loss: 0.3897 - accuracy: 0.8635 - val_loss: 0.2958 - val_accuracy: 0.8930
Epoch 6/10
101/101 [==============================] - 12s 122ms/step - loss: 0.3489 - accuracy: 0.8740 - val_loss: 0.2812 - val_accuracy: 0.9005
Epoch 7/10
101/101 [==============================] - 12s 123ms/step - loss: 0.3375 - accuracy: 0.8834 - val_loss: 0.2819 - val

In [31]:
#
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

19
Epoch 1/10
110/110 [==============================] - 15s 122ms/step - loss: 0.5618 - accuracy: 0.8008 - val_loss: 0.2783 - val_accuracy: 0.8936
Epoch 2/10
110/110 [==============================] - 13s 119ms/step - loss: 0.4472 - accuracy: 0.8368 - val_loss: 0.2785 - val_accuracy: 0.8941
Epoch 3/10
110/110 [==============================] - 13s 120ms/step - loss: 0.4217 - accuracy: 0.8474 - val_loss: 0.2831 - val_accuracy: 0.8964
Epoch 4/10
110/110 [==============================] - 13s 120ms/step - loss: 0.3814 - accuracy: 0.8656 - val_loss: 0.2811 - val_accuracy: 0.8969
Epoch 5/10
110/110 [==============================] - 13s 120ms/step - loss: 0.3585 - accuracy: 0.8773 - val_loss: 0.2865 - val_accuracy: 0.8964
Epoch 6/10
110/110 [==============================] - 13s 119ms/step - loss: 0.3351 - accuracy: 0.8798 - val_loss: 0.3146 - val_accuracy: 0.8929
Epoch 7/10
110/110 [==============================] - 13s 119ms/step - loss: 0.3146 - accuracy: 0.8911 - val_loss: 0.2924 - val

In [32]:
#new_model_dg_dir  = "C:/Users/fjdurlop/Documents/upc/models/"+dataset+"/C3/"+dataset+"_model_c3_aug_dg_e1"
new_model_dg_dir  = "D:/models/aug_22/"+dataset+"/C3/"+dataset+"_model_c3_may_dg_e1"

i=11

for model in models_dg[i:]:
    model.save(new_model_dg_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_may_dg_e1_11\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_may_dg_e1_12\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_may_dg_e1_13\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_may_dg_e1_14\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_may_dg_e1_15\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_may_dg_e1_16\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_may_dg_e1_17\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_may_dg_e1_18\assets
Model has been saved
INFO:tensorflow:Assets written t

In [33]:
try:
    del deep_gini_values
    del top_images_by_dg
    del top_labels_by_dg
    del image_sets_dg
    del label_sets_dg
    del models_dg
except:
    print("Error")

In [34]:

gc.collect()

302604

### Softmax 

In [35]:
#softmax values
#se_direction = "C:/Users/fjdurlop/Documents/upc/upc-july/data/"+dataset+"/softmax_values.npy"
se_direction = "D:/guided-retraining/data/"+dataset+"/softmax_values.npy"

se_values = np.load(se_direction)[len(x_train):] 
print(len(se_values))


6999


In [36]:
# Obtaining top n images by LSA values
top_images_by_se  = utils.get_x_of_indexes(list(np.flip(np.argsort(se_values))),x_adversary_training)
top_labels_by_se = utils.get_x_of_indexes(list(np.flip(np.argsort(se_values))),y_adversary_training)

In [37]:
m = n_data_points

n = 0
image_sets_se = []
label_sets_se = []

# last
#for i in range(0,len(top_images_by_lsa)//m):

for i in range((len(top_images_by_se)//m)):
    print(i,":")
    if (i+1 >= ((len(top_images_by_se)//m))):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_se)%m))
        top_images_by_se_n = np.array(top_images_by_se[:n+m+(len(top_images_by_se)%m)])
        top_labels_by_se_n = np.array(top_labels_by_se[:n+m+(len(top_images_by_se)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_se_n = np.array(top_images_by_se[:n+m])
        top_labels_by_se_n = np.array(top_labels_by_se[:n+m])
    image_sets_se.append(top_images_by_se_n)
    label_sets_se.append(top_labels_by_se_n)
    print(len(top_images_by_se_n))
    n += m



0 :
0  ->  340
340
1 :
0  ->  680
680
2 :
0  ->  1020
1020
3 :
0  ->  1360
1360
4 :
0  ->  1700
1700
5 :
0  ->  2040
2040
6 :
0  ->  2380
2380
7 :
0  ->  2720
2720
8 :
0  ->  3060
3060
9 :
0  ->  3400
3400
10 :
0  ->  3740
3740
11 :
0  ->  4080
4080
12 :
0  ->  4420
4420
13 :
0  ->  4760
4760
14 :
0  ->  5100
5100
15 :
0  ->  5440
5440
16 :
0  ->  5780
5780
17 :
0  ->  6120
6120
18 :
0  ->  6460
6460
19 :
Last
0  ->  6999
6999


In [38]:
print(model_dir)
models_se = []
for i in range(len(label_sets_se)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_se.append(model)

D:/guided-retraining/models/model_fashion_2
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [39]:
n=0
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

0
Epoch 1/10
6/6 [==============================] - 7s 1s/step - loss: 1.4763 - accuracy: 0.4676 - val_loss: 0.2879 - val_accuracy: 0.8996
Epoch 2/10
6/6 [==============================] - 5s 1s/step - loss: 0.9949 - accuracy: 0.6794 - val_loss: 0.3045 - val_accuracy: 0.8944
Epoch 3/10
6/6 [==============================] - 5s 1s/step - loss: 0.9150 - accuracy: 0.6765 - val_loss: 0.3101 - val_accuracy: 0.8930
Epoch 4/10
6/6 [==============================] - 5s 993ms/step - loss: 0.7787 - accuracy: 0.7000 - val_loss: 0.3202 - val_accuracy: 0.8935
Epoch 5/10
6/6 [==============================] - 4s 834ms/step - loss: 0.7254 - accuracy: 0.7382 - val_loss: 0.3490 - val_accuracy: 0.8855
Epoch 6/10
6/6 [==============================] - 4s 853ms/step - loss: 0.6252 - accuracy: 0.7765 - val_loss: 0.3458 - val_accuracy: 0.8871
Epoch 7/10
6/6 [==============================] - 4s 803ms/step - loss: 0.5410 - accuracy: 0.7941 - val_loss: 0.3511 - val_accuracy: 0.8876
Epoch 8/10
6/6 [===========

In [40]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

1
Epoch 1/10
11/11 [==============================] - 6s 489ms/step - loss: 1.3165 - accuracy: 0.5059 - val_loss: 0.2896 - val_accuracy: 0.8953
Epoch 2/10
11/11 [==============================] - 5s 478ms/step - loss: 0.8922 - accuracy: 0.6426 - val_loss: 0.3031 - val_accuracy: 0.8938
Epoch 3/10
11/11 [==============================] - 5s 507ms/step - loss: 0.8102 - accuracy: 0.7015 - val_loss: 0.3204 - val_accuracy: 0.8881
Epoch 4/10
11/11 [==============================] - 5s 453ms/step - loss: 0.7684 - accuracy: 0.7206 - val_loss: 0.3127 - val_accuracy: 0.8971
Epoch 5/10
11/11 [==============================] - 5s 449ms/step - loss: 0.6811 - accuracy: 0.7412 - val_loss: 0.3201 - val_accuracy: 0.8966
Epoch 6/10
11/11 [==============================] - 4s 405ms/step - loss: 0.6375 - accuracy: 0.7441 - val_loss: 0.3466 - val_accuracy: 0.8906
Epoch 7/10
11/11 [==============================] - 4s 398ms/step - loss: 0.6279 - accuracy: 0.7500 - val_loss: 0.3455 - val_accuracy: 0.8925
Epoc

In [41]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

2
Epoch 1/10
16/16 [==============================] - 5s 286ms/step - loss: 1.1948 - accuracy: 0.5647 - val_loss: 0.2728 - val_accuracy: 0.9040
Epoch 2/10
16/16 [==============================] - 4s 280ms/step - loss: 0.9396 - accuracy: 0.6441 - val_loss: 0.2811 - val_accuracy: 0.9024
Epoch 3/10
16/16 [==============================] - 4s 271ms/step - loss: 0.8695 - accuracy: 0.6775 - val_loss: 0.2873 - val_accuracy: 0.9001
Epoch 4/10
16/16 [==============================] - 4s 267ms/step - loss: 0.7524 - accuracy: 0.7059 - val_loss: 0.3085 - val_accuracy: 0.8982
Epoch 5/10
16/16 [==============================] - 4s 261ms/step - loss: 0.7399 - accuracy: 0.7284 - val_loss: 0.3175 - val_accuracy: 0.8951
Epoch 6/10
16/16 [==============================] - 4s 242ms/step - loss: 0.6759 - accuracy: 0.7353 - val_loss: 0.3326 - val_accuracy: 0.8951
Epoch 7/10
16/16 [==============================] - 4s 247ms/step - loss: 0.6242 - accuracy: 0.7578 - val_loss: 0.3303 - val_accuracy: 0.8951
Epoc

In [42]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

3
Epoch 1/10
22/22 [==============================] - 5s 194ms/step - loss: 1.2128 - accuracy: 0.5294 - val_loss: 0.2800 - val_accuracy: 0.8973
Epoch 2/10
22/22 [==============================] - 4s 183ms/step - loss: 0.9311 - accuracy: 0.6346 - val_loss: 0.2840 - val_accuracy: 0.8954
Epoch 3/10
22/22 [==============================] - 4s 184ms/step - loss: 0.8427 - accuracy: 0.6640 - val_loss: 0.2959 - val_accuracy: 0.8959
Epoch 4/10
22/22 [==============================] - 4s 186ms/step - loss: 0.7571 - accuracy: 0.7029 - val_loss: 0.2987 - val_accuracy: 0.8974
Epoch 5/10
22/22 [==============================] - 4s 185ms/step - loss: 0.7059 - accuracy: 0.7353 - val_loss: 0.3047 - val_accuracy: 0.8962
Epoch 6/10
22/22 [==============================] - 4s 184ms/step - loss: 0.6821 - accuracy: 0.7338 - val_loss: 0.3134 - val_accuracy: 0.8989
Epoch 7/10
22/22 [==============================] - 4s 185ms/step - loss: 0.6855 - accuracy: 0.7287 - val_loss: 0.3114 - val_accuracy: 0.8971
Epoc

In [43]:
#
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

4
Epoch 1/10
27/27 [==============================] - 5s 166ms/step - loss: 1.1311 - accuracy: 0.5671 - val_loss: 0.2739 - val_accuracy: 0.9019
Epoch 2/10
27/27 [==============================] - 4s 160ms/step - loss: 0.9212 - accuracy: 0.6324 - val_loss: 0.2725 - val_accuracy: 0.9007
Epoch 3/10
27/27 [==============================] - 4s 159ms/step - loss: 0.8086 - accuracy: 0.6506 - val_loss: 0.2750 - val_accuracy: 0.9014
Epoch 4/10
27/27 [==============================] - 4s 160ms/step - loss: 0.7806 - accuracy: 0.6994 - val_loss: 0.2889 - val_accuracy: 0.8976
Epoch 5/10
27/27 [==============================] - 4s 162ms/step - loss: 0.7267 - accuracy: 0.7141 - val_loss: 0.2895 - val_accuracy: 0.8980
Epoch 6/10
27/27 [==============================] - 4s 160ms/step - loss: 0.6742 - accuracy: 0.7253 - val_loss: 0.2932 - val_accuracy: 0.8984
Epoch 7/10
27/27 [==============================] - 4s 160ms/step - loss: 0.6577 - accuracy: 0.7518 - val_loss: 0.2836 - val_accuracy: 0.9021
Epoc

In [44]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

5
Epoch 1/10
32/32 [==============================] - 5s 149ms/step - loss: 1.0759 - accuracy: 0.5613 - val_loss: 0.2791 - val_accuracy: 0.8952
Epoch 2/10
32/32 [==============================] - 5s 144ms/step - loss: 0.8939 - accuracy: 0.6574 - val_loss: 0.2804 - val_accuracy: 0.8934
Epoch 3/10
32/32 [==============================] - 5s 146ms/step - loss: 0.7957 - accuracy: 0.6838 - val_loss: 0.2847 - val_accuracy: 0.8939
Epoch 4/10
32/32 [==============================] - 5s 150ms/step - loss: 0.7302 - accuracy: 0.6975 - val_loss: 0.2852 - val_accuracy: 0.8986
Epoch 5/10
32/32 [==============================] - 5s 151ms/step - loss: 0.6876 - accuracy: 0.7225 - val_loss: 0.2736 - val_accuracy: 0.9041
Epoch 6/10
32/32 [==============================] - 5s 150ms/step - loss: 0.6600 - accuracy: 0.7475 - val_loss: 0.2862 - val_accuracy: 0.9024
Epoch 7/10
32/32 [==============================] - 5s 150ms/step - loss: 0.6234 - accuracy: 0.7569 - val_loss: 0.2978 - val_accuracy: 0.8989
Epoc

In [45]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

6
Epoch 1/10
38/38 [==============================] - 6s 140ms/step - loss: 1.0644 - accuracy: 0.5697 - val_loss: 0.2797 - val_accuracy: 0.8933
Epoch 2/10
38/38 [==============================] - 5s 134ms/step - loss: 0.8630 - accuracy: 0.6580 - val_loss: 0.2723 - val_accuracy: 0.8960
Epoch 3/10
38/38 [==============================] - 5s 135ms/step - loss: 0.8049 - accuracy: 0.6945 - val_loss: 0.2783 - val_accuracy: 0.8948
Epoch 4/10
38/38 [==============================] - 5s 133ms/step - loss: 0.7151 - accuracy: 0.7265 - val_loss: 0.2743 - val_accuracy: 0.9029
Epoch 5/10
38/38 [==============================] - 5s 134ms/step - loss: 0.6951 - accuracy: 0.7315 - val_loss: 0.2802 - val_accuracy: 0.8991
Epoch 6/10
38/38 [==============================] - 5s 133ms/step - loss: 0.6656 - accuracy: 0.7483 - val_loss: 0.3054 - val_accuracy: 0.8911
Epoch 7/10
38/38 [==============================] - 5s 139ms/step - loss: 0.6286 - accuracy: 0.7500 - val_loss: 0.2846 - val_accuracy: 0.8995
Epoc

In [46]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

7
Epoch 1/10
43/43 [==============================] - 8s 150ms/step - loss: 1.0242 - accuracy: 0.5923 - val_loss: 0.2749 - val_accuracy: 0.8991
Epoch 2/10
43/43 [==============================] - 7s 162ms/step - loss: 0.8450 - accuracy: 0.6728 - val_loss: 0.2803 - val_accuracy: 0.8969
Epoch 3/10
43/43 [==============================] - 6s 132ms/step - loss: 0.7465 - accuracy: 0.7018 - val_loss: 0.2685 - val_accuracy: 0.9007
Epoch 4/10
43/43 [==============================] - 6s 133ms/step - loss: 0.7207 - accuracy: 0.7210 - val_loss: 0.2720 - val_accuracy: 0.9017
Epoch 5/10
43/43 [==============================] - 6s 150ms/step - loss: 0.6577 - accuracy: 0.7474 - val_loss: 0.2831 - val_accuracy: 0.8979
Epoch 6/10
43/43 [==============================] - 6s 135ms/step - loss: 0.6410 - accuracy: 0.7467 - val_loss: 0.2858 - val_accuracy: 0.8981
Epoch 7/10
43/43 [==============================] - 6s 145ms/step - loss: 0.6277 - accuracy: 0.7533 - val_loss: 0.2805 - val_accuracy: 0.8986
Epoc

In [47]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

8
Epoch 1/10
48/48 [==============================] - 6s 123ms/step - loss: 1.0109 - accuracy: 0.6029 - val_loss: 0.2670 - val_accuracy: 0.9019
Epoch 2/10
48/48 [==============================] - 6s 118ms/step - loss: 0.8195 - accuracy: 0.6859 - val_loss: 0.2656 - val_accuracy: 0.9004
Epoch 3/10
48/48 [==============================] - 6s 118ms/step - loss: 0.7400 - accuracy: 0.7193 - val_loss: 0.2713 - val_accuracy: 0.8972
Epoch 4/10
48/48 [==============================] - 6s 118ms/step - loss: 0.6895 - accuracy: 0.7386 - val_loss: 0.2770 - val_accuracy: 0.8977
Epoch 5/10
48/48 [==============================] - 6s 118ms/step - loss: 0.6702 - accuracy: 0.7415 - val_loss: 0.2820 - val_accuracy: 0.8947
Epoch 6/10
48/48 [==============================] - 6s 119ms/step - loss: 0.6139 - accuracy: 0.7605 - val_loss: 0.2759 - val_accuracy: 0.9021
Epoch 7/10
48/48 [==============================] - 6s 118ms/step - loss: 0.5959 - accuracy: 0.7729 - val_loss: 0.2774 - val_accuracy: 0.9003
Epoc

In [48]:
#
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

9
Epoch 1/10
54/54 [==============================] - 7s 114ms/step - loss: 0.9966 - accuracy: 0.6224 - val_loss: 0.2669 - val_accuracy: 0.8989
Epoch 2/10
54/54 [==============================] - 6s 111ms/step - loss: 0.7883 - accuracy: 0.6891 - val_loss: 0.2661 - val_accuracy: 0.8985
Epoch 3/10
54/54 [==============================] - 6s 110ms/step - loss: 0.7143 - accuracy: 0.7309 - val_loss: 0.2922 - val_accuracy: 0.8862
Epoch 4/10
54/54 [==============================] - 6s 110ms/step - loss: 0.6772 - accuracy: 0.7435 - val_loss: 0.2908 - val_accuracy: 0.8911
Epoch 5/10
54/54 [==============================] - 6s 115ms/step - loss: 0.6389 - accuracy: 0.7638 - val_loss: 0.2905 - val_accuracy: 0.8956
Epoch 6/10
54/54 [==============================] - 6s 112ms/step - loss: 0.5813 - accuracy: 0.7791 - val_loss: 0.2875 - val_accuracy: 0.8974
Epoch 7/10
54/54 [==============================] - 6s 113ms/step - loss: 0.5581 - accuracy: 0.7906 - val_loss: 0.2745 - val_accuracy: 0.9026
Epoc

In [49]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

10
Epoch 1/10
59/59 [==============================] - 7s 109ms/step - loss: 0.9567 - accuracy: 0.6564 - val_loss: 0.2741 - val_accuracy: 0.8956
Epoch 2/10
59/59 [==============================] - 6s 107ms/step - loss: 0.7656 - accuracy: 0.7126 - val_loss: 0.2691 - val_accuracy: 0.8992
Epoch 3/10
59/59 [==============================] - 6s 109ms/step - loss: 0.6873 - accuracy: 0.7441 - val_loss: 0.2686 - val_accuracy: 0.8983
Epoch 4/10
59/59 [==============================] - 6s 109ms/step - loss: 0.6433 - accuracy: 0.7572 - val_loss: 0.2710 - val_accuracy: 0.9018
Epoch 5/10
59/59 [==============================] - 6s 110ms/step - loss: 0.6175 - accuracy: 0.7642 - val_loss: 0.2847 - val_accuracy: 0.8962
Epoch 6/10
59/59 [==============================] - 6s 106ms/step - loss: 0.5669 - accuracy: 0.7941 - val_loss: 0.2784 - val_accuracy: 0.8999
Epoch 7/10
59/59 [==============================] - 6s 106ms/step - loss: 0.5339 - accuracy: 0.8035 - val_loss: 0.2762 - val_accuracy: 0.9004
Epo

In [50]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

11
Epoch 1/10
64/64 [==============================] - 7s 105ms/step - loss: 0.8781 - accuracy: 0.6846 - val_loss: 0.2746 - val_accuracy: 0.8982
Epoch 2/10
64/64 [==============================] - 7s 105ms/step - loss: 0.7208 - accuracy: 0.7299 - val_loss: 0.2768 - val_accuracy: 0.8948
Epoch 3/10
64/64 [==============================] - 7s 110ms/step - loss: 0.6771 - accuracy: 0.7605 - val_loss: 0.2704 - val_accuracy: 0.9003
Epoch 4/10
64/64 [==============================] - 7s 106ms/step - loss: 0.6129 - accuracy: 0.7711 - val_loss: 0.2762 - val_accuracy: 0.8986
Epoch 5/10
64/64 [==============================] - 7s 107ms/step - loss: 0.5677 - accuracy: 0.7848 - val_loss: 0.2812 - val_accuracy: 0.8969
Epoch 6/10
64/64 [==============================] - 7s 106ms/step - loss: 0.5403 - accuracy: 0.8051 - val_loss: 0.2730 - val_accuracy: 0.9001
Epoch 7/10
64/64 [==============================] - 7s 108ms/step - loss: 0.5030 - accuracy: 0.8213 - val_loss: 0.2798 - val_accuracy: 0.9008
Epo

In [51]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

12
Epoch 1/10
70/70 [==============================] - 8s 104ms/step - loss: 0.8499 - accuracy: 0.6898 - val_loss: 0.2875 - val_accuracy: 0.8884
Epoch 2/10
70/70 [==============================] - 7s 104ms/step - loss: 0.7251 - accuracy: 0.7303 - val_loss: 0.2872 - val_accuracy: 0.8938
Epoch 3/10
70/70 [==============================] - 7s 103ms/step - loss: 0.6384 - accuracy: 0.7633 - val_loss: 0.2801 - val_accuracy: 0.8918
Epoch 4/10
70/70 [==============================] - 7s 103ms/step - loss: 0.5891 - accuracy: 0.7846 - val_loss: 0.2841 - val_accuracy: 0.8931
Epoch 5/10
70/70 [==============================] - 7s 103ms/step - loss: 0.5634 - accuracy: 0.7968 - val_loss: 0.2767 - val_accuracy: 0.8978
Epoch 6/10
70/70 [==============================] - 7s 104ms/step - loss: 0.5130 - accuracy: 0.8072 - val_loss: 0.2962 - val_accuracy: 0.8976
Epoch 7/10
70/70 [==============================] - 7s 104ms/step - loss: 0.5048 - accuracy: 0.8140 - val_loss: 0.2922 - val_accuracy: 0.8915
Epo

In [52]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

13
Epoch 1/10
75/75 [==============================] - 8s 103ms/step - loss: 0.8137 - accuracy: 0.7078 - val_loss: 0.2871 - val_accuracy: 0.8912
Epoch 2/10
75/75 [==============================] - 8s 103ms/step - loss: 0.6637 - accuracy: 0.7590 - val_loss: 0.2838 - val_accuracy: 0.8927
Epoch 3/10
75/75 [==============================] - 8s 104ms/step - loss: 0.5908 - accuracy: 0.7838 - val_loss: 0.2720 - val_accuracy: 0.8975
Epoch 4/10
75/75 [==============================] - 8s 105ms/step - loss: 0.5585 - accuracy: 0.7971 - val_loss: 0.2727 - val_accuracy: 0.8966
Epoch 5/10
75/75 [==============================] - 8s 104ms/step - loss: 0.5194 - accuracy: 0.8090 - val_loss: 0.2795 - val_accuracy: 0.8946
Epoch 6/10
75/75 [==============================] - 8s 104ms/step - loss: 0.4974 - accuracy: 0.8183 - val_loss: 0.2751 - val_accuracy: 0.9017
Epoch 7/10
75/75 [==============================] - 8s 104ms/step - loss: 0.4595 - accuracy: 0.8313 - val_loss: 0.2725 - val_accuracy: 0.9027
Epo

In [53]:
#
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

14
Epoch 1/10
80/80 [==============================] - 9s 102ms/step - loss: 0.7589 - accuracy: 0.7267 - val_loss: 0.2704 - val_accuracy: 0.8986
Epoch 2/10
80/80 [==============================] - 8s 102ms/step - loss: 0.6134 - accuracy: 0.7747 - val_loss: 0.2923 - val_accuracy: 0.8902
Epoch 3/10
80/80 [==============================] - 8s 100ms/step - loss: 0.5466 - accuracy: 0.7945 - val_loss: 0.2904 - val_accuracy: 0.8919
Epoch 4/10
80/80 [==============================] - 8s 101ms/step - loss: 0.5262 - accuracy: 0.8094 - val_loss: 0.2821 - val_accuracy: 0.8957
Epoch 5/10
80/80 [==============================] - 8s 101ms/step - loss: 0.4813 - accuracy: 0.8227 - val_loss: 0.2777 - val_accuracy: 0.9011
Epoch 6/10
80/80 [==============================] - 8s 101ms/step - loss: 0.4507 - accuracy: 0.8429 - val_loss: 0.2741 - val_accuracy: 0.8994
Epoch 7/10
80/80 [==============================] - 8s 102ms/step - loss: 0.4216 - accuracy: 0.8488 - val_loss: 0.2866 - val_accuracy: 0.9025
Epo

In [54]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

15
Epoch 1/10
85/85 [==============================] - 9s 100ms/step - loss: 0.7072 - accuracy: 0.7447 - val_loss: 0.2911 - val_accuracy: 0.8945
Epoch 2/10
85/85 [==============================] - 8s 99ms/step - loss: 0.5615 - accuracy: 0.7869 - val_loss: 0.2838 - val_accuracy: 0.8944
Epoch 3/10
85/85 [==============================] - 8s 98ms/step - loss: 0.5274 - accuracy: 0.8081 - val_loss: 0.2779 - val_accuracy: 0.8969
Epoch 4/10
85/85 [==============================] - 8s 98ms/step - loss: 0.4921 - accuracy: 0.8221 - val_loss: 0.2783 - val_accuracy: 0.8979
Epoch 5/10
85/85 [==============================] - 8s 98ms/step - loss: 0.4590 - accuracy: 0.8324 - val_loss: 0.2758 - val_accuracy: 0.9014
Epoch 6/10
85/85 [==============================] - 8s 99ms/step - loss: 0.4391 - accuracy: 0.8456 - val_loss: 0.2848 - val_accuracy: 0.8977
Epoch 7/10
85/85 [==============================] - 8s 98ms/step - loss: 0.4018 - accuracy: 0.8557 - val_loss: 0.2889 - val_accuracy: 0.8986
Epoch 8/1

In [55]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

16
Epoch 1/10
91/91 [==============================] - 9s 96ms/step - loss: 0.6693 - accuracy: 0.7557 - val_loss: 0.2806 - val_accuracy: 0.8905
Epoch 2/10
91/91 [==============================] - 9s 97ms/step - loss: 0.5479 - accuracy: 0.7998 - val_loss: 0.2883 - val_accuracy: 0.8898
Epoch 3/10
91/91 [==============================] - 9s 96ms/step - loss: 0.4960 - accuracy: 0.8202 - val_loss: 0.2828 - val_accuracy: 0.8939
Epoch 4/10
91/91 [==============================] - 9s 96ms/step - loss: 0.4580 - accuracy: 0.8336 - val_loss: 0.2822 - val_accuracy: 0.8960
Epoch 5/10
91/91 [==============================] - 9s 96ms/step - loss: 0.4436 - accuracy: 0.8426 - val_loss: 0.2773 - val_accuracy: 0.8966
Epoch 6/10
91/91 [==============================] - 9s 96ms/step - loss: 0.4047 - accuracy: 0.8533 - val_loss: 0.3104 - val_accuracy: 0.8922
Epoch 7/10
91/91 [==============================] - 9s 96ms/step - loss: 0.3922 - accuracy: 0.8599 - val_loss: 0.2882 - val_accuracy: 0.8990
Epoch 8/10

In [56]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

17
Epoch 1/10
96/96 [==============================] - 10s 95ms/step - loss: 0.6324 - accuracy: 0.7717 - val_loss: 0.2835 - val_accuracy: 0.8935
Epoch 2/10
96/96 [==============================] - 9s 95ms/step - loss: 0.5206 - accuracy: 0.8111 - val_loss: 0.2698 - val_accuracy: 0.9000
Epoch 3/10
96/96 [==============================] - 10s 100ms/step - loss: 0.4709 - accuracy: 0.8310 - val_loss: 0.2804 - val_accuracy: 0.8946
Epoch 4/10
96/96 [==============================] - 9s 93ms/step - loss: 0.4244 - accuracy: 0.8474 - val_loss: 0.2728 - val_accuracy: 0.9019
Epoch 5/10
96/96 [==============================] - 9s 93ms/step - loss: 0.3976 - accuracy: 0.8577 - val_loss: 0.2785 - val_accuracy: 0.9021
Epoch 6/10
96/96 [==============================] - 9s 95ms/step - loss: 0.3837 - accuracy: 0.8655 - val_loss: 0.2782 - val_accuracy: 0.9013
Epoch 7/10
96/96 [==============================] - 9s 94ms/step - loss: 0.3456 - accuracy: 0.8778 - val_loss: 0.3006 - val_accuracy: 0.8933
Epoch 8

In [57]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

18
Epoch 1/10
101/101 [==============================] - 10s 94ms/step - loss: 0.6036 - accuracy: 0.7881 - val_loss: 0.2780 - val_accuracy: 0.8958
Epoch 2/10
101/101 [==============================] - 9s 91ms/step - loss: 0.4902 - accuracy: 0.8217 - val_loss: 0.2789 - val_accuracy: 0.8999
Epoch 3/10
101/101 [==============================] - 9s 92ms/step - loss: 0.4384 - accuracy: 0.8430 - val_loss: 0.2806 - val_accuracy: 0.8933
Epoch 4/10
101/101 [==============================] - 9s 92ms/step - loss: 0.4033 - accuracy: 0.8565 - val_loss: 0.2874 - val_accuracy: 0.8983
Epoch 5/10
101/101 [==============================] - 9s 92ms/step - loss: 0.3828 - accuracy: 0.8639 - val_loss: 0.2905 - val_accuracy: 0.8984
Epoch 6/10
101/101 [==============================] - 9s 94ms/step - loss: 0.3676 - accuracy: 0.8715 - val_loss: 0.2821 - val_accuracy: 0.9019
Epoch 7/10
101/101 [==============================] - 10s 97ms/step - loss: 0.3397 - accuracy: 0.8807 - val_loss: 0.2847 - val_accuracy: 0

In [58]:
#
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

19
Epoch 1/10
110/110 [==============================] - 11s 90ms/step - loss: 0.5467 - accuracy: 0.8004 - val_loss: 0.2810 - val_accuracy: 0.8983
Epoch 2/10
110/110 [==============================] - 10s 88ms/step - loss: 0.4505 - accuracy: 0.8384 - val_loss: 0.2879 - val_accuracy: 0.8951
Epoch 3/10
110/110 [==============================] - 10s 89ms/step - loss: 0.4235 - accuracy: 0.8537 - val_loss: 0.2790 - val_accuracy: 0.8994
Epoch 4/10
110/110 [==============================] - 10s 89ms/step - loss: 0.3835 - accuracy: 0.8644 - val_loss: 0.2791 - val_accuracy: 0.8965
Epoch 5/10
110/110 [==============================] - 10s 88ms/step - loss: 0.3557 - accuracy: 0.8751 - val_loss: 0.3057 - val_accuracy: 0.8895
Epoch 6/10
110/110 [==============================] - 10s 90ms/step - loss: 0.3303 - accuracy: 0.8837 - val_loss: 0.3084 - val_accuracy: 0.8942
Epoch 7/10
110/110 [==============================] - 10s 88ms/step - loss: 0.3100 - accuracy: 0.8891 - val_loss: 0.2914 - val_accura

In [59]:
#new_model_se_dir  = "C:/Users/fjdurlop/Documents/upc/models/"+dataset+"/C3/"+dataset+"_model_c3_aug_se_e1"
new_model_se_dir  = "D:/models/aug_22/"+dataset+"/C3/"+dataset+"_model_c3_may_se_e1"

i=0

for model in models_se:
    model.save(new_model_se_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_may_se_e1_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_may_se_e1_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_may_se_e1_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_may_se_e1_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_may_se_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_may_se_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_may_se_e1_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_may_se_e1_7\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/mo

In [60]:
try:
    del se_values
    del top_images_by_se
    del top_labels_by_se
    del image_sets_se
    del label_sets_se
    del models_se
except:
    print("Error")

In [61]:
gc.collect()

349909

## Training guided by Random values

In [62]:
#save_dir = "C:/Users/fjdurlop/Documents/upc/upc-july/data/"+dataset+"/random_values.npy"
save_dir = "D:/guided-retraining/data/"+dataset+"/random_values.npy"


random_indexes = np.load(save_dir)[len(x_train):] 
print(len(random_indexes))

6999


In [63]:
len(random_indexes)

6999

In [64]:
# Obtaining top n images by random values
top_images_by_random = utils.get_x_of_indexes(list(np.flip(np.argsort(random_indexes))),x_adversary_training)
top_labels_by_random = utils.get_x_of_indexes(list(np.flip(np.argsort(random_indexes))),y_adversary_training)

In [65]:
m = n_data_points
n = 0
image_sets_random = []
label_sets_random = []



for i in range((len(top_images_by_random)//m)):
    print(i,":")
    if (i+1 >= ((len(top_images_by_random)//m))):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_random)%m))
        top_images_by_random_n = np.array(top_images_by_random[:n+m+(len(top_images_by_random)%m)])
        top_labels_by_random_n = np.array(top_labels_by_random[:n+m+(len(top_images_by_random)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_random_n = np.array(top_images_by_random[:n+m])
        top_labels_by_random_n = np.array(top_labels_by_random[:n+m])
    image_sets_random.append(top_images_by_random_n)
    label_sets_random.append(top_labels_by_random_n)
    print(len(top_images_by_random_n))
    n += m




0 :
0  ->  340
340
1 :
0  ->  680
680
2 :
0  ->  1020
1020
3 :
0  ->  1360
1360
4 :
0  ->  1700
1700
5 :
0  ->  2040
2040
6 :
0  ->  2380
2380
7 :
0  ->  2720
2720
8 :
0  ->  3060
3060
9 :
0  ->  3400
3400
10 :
0  ->  3740
3740
11 :
0  ->  4080
4080
12 :
0  ->  4420
4420
13 :
0  ->  4760
4760
14 :
0  ->  5100
5100
15 :
0  ->  5440
5440
16 :
0  ->  5780
5780
17 :
0  ->  6120
6120
18 :
0  ->  6460
6460
19 :
Last
0  ->  6999
6999


In [66]:
len(image_sets_random)

20

In [67]:
print(model_dir)

models_random = []
for i in range(len(label_sets_random)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_random.append(model)


D:/guided-retraining/models/model_fashion_2
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [68]:
n=0
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

0
Epoch 1/10
6/6 [==============================] - 5s 851ms/step - loss: 0.7111 - accuracy: 0.7824 - val_loss: 0.2652 - val_accuracy: 0.9057
Epoch 2/10
6/6 [==============================] - 4s 822ms/step - loss: 0.5227 - accuracy: 0.8176 - val_loss: 0.2576 - val_accuracy: 0.9064
Epoch 3/10
6/6 [==============================] - 5s 890ms/step - loss: 0.4604 - accuracy: 0.8471 - val_loss: 0.2636 - val_accuracy: 0.9074
Epoch 4/10
6/6 [==============================] - 4s 870ms/step - loss: 0.4168 - accuracy: 0.8559 - val_loss: 0.2781 - val_accuracy: 0.9019
Epoch 5/10
6/6 [==============================] - 5s 887ms/step - loss: 0.3861 - accuracy: 0.8559 - val_loss: 0.2714 - val_accuracy: 0.9034
Epoch 6/10
6/6 [==============================] - 5s 893ms/step - loss: 0.3553 - accuracy: 0.8559 - val_loss: 0.2841 - val_accuracy: 0.8992
Epoch 7/10
6/6 [==============================] - 4s 828ms/step - loss: 0.3095 - accuracy: 0.8824 - val_loss: 0.2844 - val_accuracy: 0.9009
Epoch 8/10
6/6 [==

In [69]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

1
Epoch 1/10
11/11 [==============================] - 6s 492ms/step - loss: 0.6658 - accuracy: 0.7779 - val_loss: 0.2640 - val_accuracy: 0.9060
Epoch 2/10
11/11 [==============================] - 5s 474ms/step - loss: 0.5346 - accuracy: 0.8088 - val_loss: 0.2619 - val_accuracy: 0.9014
Epoch 3/10
11/11 [==============================] - 5s 481ms/step - loss: 0.4868 - accuracy: 0.8191 - val_loss: 0.2779 - val_accuracy: 0.9019
Epoch 4/10
11/11 [==============================] - 5s 477ms/step - loss: 0.4697 - accuracy: 0.8162 - val_loss: 0.2797 - val_accuracy: 0.8984
Epoch 5/10
11/11 [==============================] - 5s 474ms/step - loss: 0.4252 - accuracy: 0.8324 - val_loss: 0.2752 - val_accuracy: 0.9008
Epoch 6/10
11/11 [==============================] - 5s 489ms/step - loss: 0.3853 - accuracy: 0.8368 - val_loss: 0.2785 - val_accuracy: 0.8967
Epoch 7/10
11/11 [==============================] - 5s 484ms/step - loss: 0.3724 - accuracy: 0.8544 - val_loss: 0.2756 - val_accuracy: 0.9033
Epoc

In [70]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

2
Epoch 1/10
16/16 [==============================] - 6s 356ms/step - loss: 0.6825 - accuracy: 0.7873 - val_loss: 0.2594 - val_accuracy: 0.9061
Epoch 2/10
16/16 [==============================] - 5s 343ms/step - loss: 0.5504 - accuracy: 0.8088 - val_loss: 0.2717 - val_accuracy: 0.8979
Epoch 3/10
16/16 [==============================] - 5s 338ms/step - loss: 0.5068 - accuracy: 0.8167 - val_loss: 0.2657 - val_accuracy: 0.9015
Epoch 4/10
16/16 [==============================] - 5s 345ms/step - loss: 0.4632 - accuracy: 0.8314 - val_loss: 0.2807 - val_accuracy: 0.8981
Epoch 5/10
16/16 [==============================] - 5s 346ms/step - loss: 0.4029 - accuracy: 0.8549 - val_loss: 0.2730 - val_accuracy: 0.9024
Epoch 6/10
16/16 [==============================] - 5s 342ms/step - loss: 0.3849 - accuracy: 0.8471 - val_loss: 0.3017 - val_accuracy: 0.8971
Epoch 7/10
16/16 [==============================] - 5s 348ms/step - loss: 0.3619 - accuracy: 0.8559 - val_loss: 0.2776 - val_accuracy: 0.9040
Epoc

In [71]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

3
Epoch 1/10
22/22 [==============================] - 7s 270ms/step - loss: 0.6771 - accuracy: 0.7699 - val_loss: 0.2765 - val_accuracy: 0.8947
Epoch 2/10
22/22 [==============================] - 6s 266ms/step - loss: 0.5521 - accuracy: 0.8051 - val_loss: 0.2830 - val_accuracy: 0.8932
Epoch 3/10
22/22 [==============================] - 5s 254ms/step - loss: 0.4845 - accuracy: 0.8243 - val_loss: 0.2773 - val_accuracy: 0.8959
Epoch 4/10
22/22 [==============================] - 5s 255ms/step - loss: 0.4444 - accuracy: 0.8404 - val_loss: 0.2842 - val_accuracy: 0.8915
Epoch 5/10
22/22 [==============================] - 6s 260ms/step - loss: 0.3972 - accuracy: 0.8537 - val_loss: 0.2871 - val_accuracy: 0.8919
Epoch 6/10
22/22 [==============================] - 6s 264ms/step - loss: 0.3885 - accuracy: 0.8632 - val_loss: 0.2921 - val_accuracy: 0.8976
Epoch 7/10
22/22 [==============================] - 6s 261ms/step - loss: 0.3728 - accuracy: 0.8676 - val_loss: 0.2910 - val_accuracy: 0.8941
Epoc

In [72]:
#
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

4
Epoch 1/10
27/27 [==============================] - 9s 270ms/step - loss: 0.6848 - accuracy: 0.7653 - val_loss: 0.2745 - val_accuracy: 0.9011
Epoch 2/10
27/27 [==============================] - 6s 245ms/step - loss: 0.5615 - accuracy: 0.8059 - val_loss: 0.2766 - val_accuracy: 0.8942
Epoch 3/10
27/27 [==============================] - 7s 250ms/step - loss: 0.4773 - accuracy: 0.8300 - val_loss: 0.2780 - val_accuracy: 0.8951
Epoch 4/10
27/27 [==============================] - 7s 252ms/step - loss: 0.4476 - accuracy: 0.8318 - val_loss: 0.2725 - val_accuracy: 0.8960
Epoch 5/10
27/27 [==============================] - 7s 254ms/step - loss: 0.4065 - accuracy: 0.8476 - val_loss: 0.2967 - val_accuracy: 0.8886
Epoch 6/10
27/27 [==============================] - 7s 254ms/step - loss: 0.3926 - accuracy: 0.8500 - val_loss: 0.2934 - val_accuracy: 0.8916
Epoch 7/10
27/27 [==============================] - 7s 248ms/step - loss: 0.3808 - accuracy: 0.8688 - val_loss: 0.2811 - val_accuracy: 0.8986
Epoc

In [73]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

5
Epoch 1/10
32/32 [==============================] - 8s 231ms/step - loss: 0.6578 - accuracy: 0.7784 - val_loss: 0.2678 - val_accuracy: 0.8979
Epoch 2/10
32/32 [==============================] - 7s 225ms/step - loss: 0.5178 - accuracy: 0.8147 - val_loss: 0.2776 - val_accuracy: 0.8931
Epoch 3/10
32/32 [==============================] - 7s 226ms/step - loss: 0.4956 - accuracy: 0.8333 - val_loss: 0.2693 - val_accuracy: 0.8991
Epoch 4/10
32/32 [==============================] - 7s 227ms/step - loss: 0.4535 - accuracy: 0.8343 - val_loss: 0.2711 - val_accuracy: 0.8987
Epoch 5/10
32/32 [==============================] - 7s 227ms/step - loss: 0.3993 - accuracy: 0.8529 - val_loss: 0.2697 - val_accuracy: 0.8996
Epoch 6/10
32/32 [==============================] - 7s 228ms/step - loss: 0.4073 - accuracy: 0.8529 - val_loss: 0.2687 - val_accuracy: 0.8993
Epoch 7/10
32/32 [==============================] - 7s 223ms/step - loss: 0.3708 - accuracy: 0.8559 - val_loss: 0.2774 - val_accuracy: 0.8986
Epoc

In [74]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

6
Epoch 1/10
38/38 [==============================] - 9s 208ms/step - loss: 0.6595 - accuracy: 0.7807 - val_loss: 0.3031 - val_accuracy: 0.8844
Epoch 2/10
38/38 [==============================] - 7s 198ms/step - loss: 0.5139 - accuracy: 0.8134 - val_loss: 0.2773 - val_accuracy: 0.9031
Epoch 3/10
38/38 [==============================] - 7s 200ms/step - loss: 0.4696 - accuracy: 0.8374 - val_loss: 0.2819 - val_accuracy: 0.8950
Epoch 4/10
38/38 [==============================] - 7s 198ms/step - loss: 0.4308 - accuracy: 0.8450 - val_loss: 0.2774 - val_accuracy: 0.8957
Epoch 5/10
38/38 [==============================] - 8s 203ms/step - loss: 0.4001 - accuracy: 0.8588 - val_loss: 0.2848 - val_accuracy: 0.8964
Epoch 6/10
38/38 [==============================] - 8s 209ms/step - loss: 0.3960 - accuracy: 0.8559 - val_loss: 0.2837 - val_accuracy: 0.8982
Epoch 7/10
38/38 [==============================] - 8s 210ms/step - loss: 0.3691 - accuracy: 0.8697 - val_loss: 0.2951 - val_accuracy: 0.8959
Epoc

In [75]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

7
Epoch 1/10
43/43 [==============================] - 10s 195ms/step - loss: 0.6386 - accuracy: 0.7757 - val_loss: 0.2827 - val_accuracy: 0.8971
Epoch 2/10
43/43 [==============================] - 8s 189ms/step - loss: 0.5393 - accuracy: 0.8099 - val_loss: 0.2823 - val_accuracy: 0.8899
Epoch 3/10
43/43 [==============================] - 8s 189ms/step - loss: 0.4650 - accuracy: 0.8401 - val_loss: 0.2805 - val_accuracy: 0.8923
Epoch 4/10
43/43 [==============================] - 8s 193ms/step - loss: 0.4510 - accuracy: 0.8313 - val_loss: 0.2877 - val_accuracy: 0.8926
Epoch 5/10
43/43 [==============================] - 8s 189ms/step - loss: 0.4160 - accuracy: 0.8511 - val_loss: 0.2787 - val_accuracy: 0.8956
Epoch 6/10
43/43 [==============================] - 8s 193ms/step - loss: 0.3882 - accuracy: 0.8592 - val_loss: 0.2930 - val_accuracy: 0.8886
Epoch 7/10
43/43 [==============================] - 8s 190ms/step - loss: 0.3772 - accuracy: 0.8588 - val_loss: 0.2792 - val_accuracy: 0.8998
Epo

In [76]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

8
Epoch 1/10
48/48 [==============================] - 11s 191ms/step - loss: 0.6252 - accuracy: 0.7804 - val_loss: 0.2777 - val_accuracy: 0.8919
Epoch 2/10
48/48 [==============================] - 9s 182ms/step - loss: 0.4967 - accuracy: 0.8229 - val_loss: 0.2702 - val_accuracy: 0.8976
Epoch 3/10
48/48 [==============================] - 9s 192ms/step - loss: 0.4361 - accuracy: 0.8412 - val_loss: 0.2787 - val_accuracy: 0.8989
Epoch 4/10
48/48 [==============================] - 9s 190ms/step - loss: 0.4185 - accuracy: 0.8546 - val_loss: 0.2870 - val_accuracy: 0.8924
Epoch 5/10
48/48 [==============================] - 9s 181ms/step - loss: 0.3922 - accuracy: 0.8592 - val_loss: 0.2755 - val_accuracy: 0.8974
Epoch 6/10
48/48 [==============================] - 8s 166ms/step - loss: 0.3717 - accuracy: 0.8758 - val_loss: 0.2856 - val_accuracy: 0.8979
Epoch 7/10
48/48 [==============================] - 8s 166ms/step - loss: 0.3718 - accuracy: 0.8709 - val_loss: 0.2836 - val_accuracy: 0.8971
Epo

In [77]:
#
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

9
Epoch 1/10
54/54 [==============================] - 9s 156ms/step - loss: 0.6294 - accuracy: 0.7850 - val_loss: 0.2749 - val_accuracy: 0.8968
Epoch 2/10
54/54 [==============================] - 8s 158ms/step - loss: 0.4969 - accuracy: 0.8165 - val_loss: 0.3076 - val_accuracy: 0.8799
Epoch 3/10
54/54 [==============================] - 8s 153ms/step - loss: 0.4461 - accuracy: 0.8359 - val_loss: 0.2706 - val_accuracy: 0.8980
Epoch 4/10
54/54 [==============================] - 8s 151ms/step - loss: 0.4282 - accuracy: 0.8432 - val_loss: 0.2778 - val_accuracy: 0.8976
Epoch 5/10
54/54 [==============================] - 8s 152ms/step - loss: 0.3727 - accuracy: 0.8588 - val_loss: 0.2825 - val_accuracy: 0.9019
Epoch 6/10
54/54 [==============================] - 8s 159ms/step - loss: 0.3810 - accuracy: 0.8621 - val_loss: 0.2850 - val_accuracy: 0.8961
Epoch 7/10
54/54 [==============================] - 8s 154ms/step - loss: 0.3634 - accuracy: 0.8674 - val_loss: 0.2796 - val_accuracy: 0.8992
Epoc

In [78]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

10
Epoch 1/10
59/59 [==============================] - 10s 152ms/step - loss: 0.6213 - accuracy: 0.7909 - val_loss: 0.2769 - val_accuracy: 0.8952
Epoch 2/10
59/59 [==============================] - 9s 152ms/step - loss: 0.4887 - accuracy: 0.8211 - val_loss: 0.2861 - val_accuracy: 0.8903
Epoch 3/10
59/59 [==============================] - 9s 151ms/step - loss: 0.4442 - accuracy: 0.8393 - val_loss: 0.2756 - val_accuracy: 0.8953
Epoch 4/10
59/59 [==============================] - 9s 148ms/step - loss: 0.4253 - accuracy: 0.8500 - val_loss: 0.2779 - val_accuracy: 0.8934
Epoch 5/10
59/59 [==============================] - 9s 151ms/step - loss: 0.3831 - accuracy: 0.8660 - val_loss: 0.2866 - val_accuracy: 0.8946
Epoch 6/10
59/59 [==============================] - 9s 149ms/step - loss: 0.3701 - accuracy: 0.8674 - val_loss: 0.2850 - val_accuracy: 0.8959
Epoch 7/10
59/59 [==============================] - 9s 147ms/step - loss: 0.3653 - accuracy: 0.8757 - val_loss: 0.2924 - val_accuracy: 0.8975
Ep

In [79]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

11
Epoch 1/10
64/64 [==============================] - 10s 146ms/step - loss: 0.5906 - accuracy: 0.7924 - val_loss: 0.2752 - val_accuracy: 0.8985
Epoch 2/10
64/64 [==============================] - 9s 143ms/step - loss: 0.4790 - accuracy: 0.8282 - val_loss: 0.2762 - val_accuracy: 0.8950
Epoch 3/10
64/64 [==============================] - 9s 143ms/step - loss: 0.4399 - accuracy: 0.8431 - val_loss: 0.2766 - val_accuracy: 0.8979
Epoch 4/10
64/64 [==============================] - 9s 145ms/step - loss: 0.4140 - accuracy: 0.8534 - val_loss: 0.2699 - val_accuracy: 0.8999
Epoch 5/10
64/64 [==============================] - 9s 146ms/step - loss: 0.3777 - accuracy: 0.8613 - val_loss: 0.2710 - val_accuracy: 0.9028
Epoch 6/10
64/64 [==============================] - 9s 140ms/step - loss: 0.3817 - accuracy: 0.8654 - val_loss: 0.2808 - val_accuracy: 0.8971
Epoch 7/10
64/64 [==============================] - 9s 139ms/step - loss: 0.3484 - accuracy: 0.8750 - val_loss: 0.2911 - val_accuracy: 0.8948
Ep

In [80]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

12
Epoch 1/10
70/70 [==============================] - 11s 140ms/step - loss: 0.6082 - accuracy: 0.7885 - val_loss: 0.2831 - val_accuracy: 0.8891
Epoch 2/10
70/70 [==============================] - 10s 138ms/step - loss: 0.4884 - accuracy: 0.8283 - val_loss: 0.2779 - val_accuracy: 0.8980
Epoch 3/10
70/70 [==============================] - 10s 137ms/step - loss: 0.4317 - accuracy: 0.8416 - val_loss: 0.2789 - val_accuracy: 0.8966
Epoch 4/10
70/70 [==============================] - 10s 137ms/step - loss: 0.4133 - accuracy: 0.8493 - val_loss: 0.2935 - val_accuracy: 0.8907
Epoch 5/10
70/70 [==============================] - 9s 135ms/step - loss: 0.3847 - accuracy: 0.8586 - val_loss: 0.2795 - val_accuracy: 0.8973
Epoch 6/10
70/70 [==============================] - 9s 136ms/step - loss: 0.3613 - accuracy: 0.8631 - val_loss: 0.2845 - val_accuracy: 0.8963
Epoch 7/10
70/70 [==============================] - 9s 136ms/step - loss: 0.3383 - accuracy: 0.8781 - val_loss: 0.3022 - val_accuracy: 0.8966

In [81]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

13
Epoch 1/10
75/75 [==============================] - 11s 132ms/step - loss: 0.5792 - accuracy: 0.7947 - val_loss: 0.2828 - val_accuracy: 0.8966
Epoch 2/10
75/75 [==============================] - 10s 131ms/step - loss: 0.4774 - accuracy: 0.8311 - val_loss: 0.2793 - val_accuracy: 0.8939
Epoch 3/10
75/75 [==============================] - 10s 132ms/step - loss: 0.4342 - accuracy: 0.8458 - val_loss: 0.2907 - val_accuracy: 0.8914
Epoch 4/10
75/75 [==============================] - 10s 132ms/step - loss: 0.4084 - accuracy: 0.8563 - val_loss: 0.3012 - val_accuracy: 0.8906
Epoch 5/10
75/75 [==============================] - 10s 135ms/step - loss: 0.3846 - accuracy: 0.8624 - val_loss: 0.2938 - val_accuracy: 0.8957
Epoch 6/10
75/75 [==============================] - 10s 133ms/step - loss: 0.3521 - accuracy: 0.8731 - val_loss: 0.3052 - val_accuracy: 0.8943
Epoch 7/10
75/75 [==============================] - 10s 137ms/step - loss: 0.3387 - accuracy: 0.8771 - val_loss: 0.2966 - val_accuracy: 0.8

In [82]:
#
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

14
Epoch 1/10
80/80 [==============================] - 11s 132ms/step - loss: 0.5804 - accuracy: 0.7963 - val_loss: 0.2822 - val_accuracy: 0.8914
Epoch 2/10
80/80 [==============================] - 10s 128ms/step - loss: 0.4726 - accuracy: 0.8296 - val_loss: 0.2782 - val_accuracy: 0.8981
Epoch 3/10
80/80 [==============================] - 10s 124ms/step - loss: 0.4355 - accuracy: 0.8437 - val_loss: 0.2751 - val_accuracy: 0.8955
Epoch 4/10
80/80 [==============================] - 10s 127ms/step - loss: 0.3956 - accuracy: 0.8649 - val_loss: 0.2795 - val_accuracy: 0.8974
Epoch 5/10
80/80 [==============================] - 10s 127ms/step - loss: 0.3844 - accuracy: 0.8641 - val_loss: 0.2875 - val_accuracy: 0.8925
Epoch 6/10
80/80 [==============================] - 10s 128ms/step - loss: 0.3541 - accuracy: 0.8753 - val_loss: 0.2835 - val_accuracy: 0.8971
Epoch 7/10
80/80 [==============================] - 10s 130ms/step - loss: 0.3321 - accuracy: 0.8804 - val_loss: 0.2870 - val_accuracy: 0.8

In [83]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

15
Epoch 1/10
85/85 [==============================] - 11s 126ms/step - loss: 0.5713 - accuracy: 0.7980 - val_loss: 0.2817 - val_accuracy: 0.8909
Epoch 2/10
85/85 [==============================] - 10s 123ms/step - loss: 0.4729 - accuracy: 0.8283 - val_loss: 0.2694 - val_accuracy: 0.8989
Epoch 3/10
85/85 [==============================] - 10s 122ms/step - loss: 0.4226 - accuracy: 0.8454 - val_loss: 0.2840 - val_accuracy: 0.8930
Epoch 4/10
85/85 [==============================] - 10s 121ms/step - loss: 0.3898 - accuracy: 0.8607 - val_loss: 0.2777 - val_accuracy: 0.8976
Epoch 5/10
85/85 [==============================] - 10s 121ms/step - loss: 0.3644 - accuracy: 0.8667 - val_loss: 0.2819 - val_accuracy: 0.8994
Epoch 6/10
85/85 [==============================] - 10s 121ms/step - loss: 0.3512 - accuracy: 0.8820 - val_loss: 0.2773 - val_accuracy: 0.9009
Epoch 7/10
85/85 [==============================] - 10s 114ms/step - loss: 0.3314 - accuracy: 0.8746 - val_loss: 0.2875 - val_accuracy: 0.9

In [84]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

16
Epoch 1/10
91/91 [==============================] - 11s 112ms/step - loss: 0.5541 - accuracy: 0.8028 - val_loss: 0.2656 - val_accuracy: 0.9004
Epoch 2/10
91/91 [==============================] - 10s 112ms/step - loss: 0.4463 - accuracy: 0.8370 - val_loss: 0.2939 - val_accuracy: 0.8904
Epoch 3/10
91/91 [==============================] - 10s 113ms/step - loss: 0.4134 - accuracy: 0.8514 - val_loss: 0.2882 - val_accuracy: 0.8911
Epoch 4/10
91/91 [==============================] - 10s 113ms/step - loss: 0.3819 - accuracy: 0.8588 - val_loss: 0.3042 - val_accuracy: 0.8891
Epoch 5/10
91/91 [==============================] - 10s 115ms/step - loss: 0.3642 - accuracy: 0.8671 - val_loss: 0.2907 - val_accuracy: 0.8981
Epoch 6/10
91/91 [==============================] - 10s 112ms/step - loss: 0.3376 - accuracy: 0.8775 - val_loss: 0.2890 - val_accuracy: 0.9002
Epoch 7/10
91/91 [==============================] - 10s 109ms/step - loss: 0.3397 - accuracy: 0.8834 - val_loss: 0.2954 - val_accuracy: 0.8

In [85]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

17
Epoch 1/10
96/96 [==============================] - 11s 107ms/step - loss: 0.5508 - accuracy: 0.8059 - val_loss: 0.2734 - val_accuracy: 0.8991
Epoch 2/10
96/96 [==============================] - 10s 107ms/step - loss: 0.4662 - accuracy: 0.8330 - val_loss: 0.2738 - val_accuracy: 0.8991
Epoch 3/10
96/96 [==============================] - 10s 107ms/step - loss: 0.4170 - accuracy: 0.8515 - val_loss: 0.2735 - val_accuracy: 0.8971
Epoch 4/10
96/96 [==============================] - 10s 107ms/step - loss: 0.3806 - accuracy: 0.8660 - val_loss: 0.2830 - val_accuracy: 0.8994
Epoch 5/10
96/96 [==============================] - 10s 108ms/step - loss: 0.3622 - accuracy: 0.8699 - val_loss: 0.2769 - val_accuracy: 0.8994
Epoch 6/10
96/96 [==============================] - 10s 110ms/step - loss: 0.3388 - accuracy: 0.8807 - val_loss: 0.2919 - val_accuracy: 0.8978
Epoch 7/10
96/96 [==============================] - 10s 106ms/step - loss: 0.3012 - accuracy: 0.8959 - val_loss: 0.2969 - val_accuracy: 0.8

In [86]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

18
Epoch 1/10
101/101 [==============================] - 12s 107ms/step - loss: 0.5596 - accuracy: 0.8014 - val_loss: 0.2902 - val_accuracy: 0.8903
Epoch 2/10
101/101 [==============================] - 11s 107ms/step - loss: 0.4595 - accuracy: 0.8367 - val_loss: 0.2814 - val_accuracy: 0.8932
Epoch 3/10
101/101 [==============================] - 11s 107ms/step - loss: 0.4225 - accuracy: 0.8498 - val_loss: 0.2761 - val_accuracy: 0.9028
Epoch 4/10
101/101 [==============================] - 11s 107ms/step - loss: 0.3900 - accuracy: 0.8601 - val_loss: 0.2820 - val_accuracy: 0.8972
Epoch 5/10
101/101 [==============================] - 11s 112ms/step - loss: 0.3637 - accuracy: 0.8732 - val_loss: 0.2776 - val_accuracy: 0.9009
Epoch 6/10
101/101 [==============================] - 11s 107ms/step - loss: 0.3478 - accuracy: 0.8793 - val_loss: 0.2868 - val_accuracy: 0.8998
Epoch 7/10
101/101 [==============================] - 11s 105ms/step - loss: 0.3052 - accuracy: 0.8912 - val_loss: 0.2870 - val

In [87]:
#
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

19
Epoch 1/10
110/110 [==============================] - 12s 101ms/step - loss: 0.5343 - accuracy: 0.8087 - val_loss: 0.2763 - val_accuracy: 0.8976
Epoch 2/10
110/110 [==============================] - 11s 102ms/step - loss: 0.4421 - accuracy: 0.8430 - val_loss: 0.3056 - val_accuracy: 0.8899
Epoch 3/10
110/110 [==============================] - 11s 101ms/step - loss: 0.3991 - accuracy: 0.8563 - val_loss: 0.2930 - val_accuracy: 0.8961
Epoch 4/10
110/110 [==============================] - 11s 103ms/step - loss: 0.3800 - accuracy: 0.8644 - val_loss: 0.2920 - val_accuracy: 0.8959
Epoch 5/10
110/110 [==============================] - 11s 103ms/step - loss: 0.3509 - accuracy: 0.8777 - val_loss: 0.2782 - val_accuracy: 0.9019
Epoch 6/10
110/110 [==============================] - 11s 103ms/step - loss: 0.3222 - accuracy: 0.8856 - val_loss: 0.3183 - val_accuracy: 0.8947
Epoch 7/10
110/110 [==============================] - 11s 103ms/step - loss: 0.3035 - accuracy: 0.8921 - val_loss: 0.3006 - val

In [88]:
#new_model_random_dir  = "C:/Users/fjdurlop/Documents/upc/models/"+dataset+"/C2/"+dataset+"_model_c2_aug_random_e1"
new_model_random_dir  = "D:/models/aug_22/"+dataset+"/C3/"+dataset+"_model_c3_aug_random_e1"

i=0

for model in models_random:
    model.save(new_model_random_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_aug_random_e1_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_aug_random_e1_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_aug_random_e1_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_aug_random_e1_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_aug_random_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_aug_random_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_aug_random_e1_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_aug_random_e1_7\assets
Model has been saved
INFO:ten

In [89]:
try:
    del random_indexes
    del top_images_by_random
    del top_labels_by_random
    del image_sets_random
    del label_sets_random
    del models_random
except:
    print("Error")

In [90]:
import gc

In [91]:
gc.collect()

405127

## Training guided by NC

In [92]:
# NC
nc_values = []
for i in range(1,17):
    #save_dir = "C:/Users/fjdur/Desktop/upc/project_notebooks/github_project/DL_notebooks/NC_values/nc_values_"+str(i)+".npy"
    #save_dir = "C:/Users/fjdurlop/Documents/upc/upc-july/data/"+dataset+"/"+dataset+"_nc_values_"+str(i)+".npy"
    save_dir = "D:/guided-retraining/data/"+dataset+"/"+dataset+"_nc_values_"+str(i)+".npy"

    #print(save_dir_rand)
    tmp_values = np.load(save_dir)
    #print(tmp_values.shape)
    nc_values = np.append(nc_values,tmp_values)
    
nc_values = nc_values[len(x_train):] 
print(len(nc_values))

6999


In [93]:
nc_values.shape

(6999,)

In [94]:

top_images_by_nc = utils.get_x_of_indexes(list(np.flip(np.argsort(nc_values))),x_adversary_training)
top_labels_by_nc = utils.get_x_of_indexes(list(np.flip(np.argsort(nc_values))),y_adversary_training)


In [95]:
m = n_data_points
n = 0
image_sets_nc = []
label_sets_nc = []


for i in range((len(top_images_by_nc)//m)):
    print(i,":")
    if (i+1 >= ((len(top_images_by_nc)//m))):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_nc)%m))
        top_images_by_nc_n = np.array(top_images_by_nc[:n+m+(len(top_images_by_nc)%m)])
        top_labels_by_nc_n = np.array(top_labels_by_nc[:n+m+(len(top_images_by_nc)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_nc_n = np.array(top_images_by_nc[:n+m])
        top_labels_by_nc_n = np.array(top_labels_by_nc[:n+m])
    image_sets_nc.append(top_images_by_nc_n)
    label_sets_nc.append(top_labels_by_nc_n)
    print(len(top_images_by_nc_n))
    n += m




0 :
0  ->  340
340
1 :
0  ->  680
680
2 :
0  ->  1020
1020
3 :
0  ->  1360
1360
4 :
0  ->  1700
1700
5 :
0  ->  2040
2040
6 :
0  ->  2380
2380
7 :
0  ->  2720
2720
8 :
0  ->  3060
3060
9 :
0  ->  3400
3400
10 :
0  ->  3740
3740
11 :
0  ->  4080
4080
12 :
0  ->  4420
4420
13 :
0  ->  4760
4760
14 :
0  ->  5100
5100
15 :
0  ->  5440
5440
16 :
0  ->  5780
5780
17 :
0  ->  6120
6120
18 :
0  ->  6460
6460
19 :
Last
0  ->  6999
6999


In [96]:
print(model_dir)

models_nc = []
for i in range(len(label_sets_nc)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_nc.append(model)


D:/guided-retraining/models/model_fashion_2
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [97]:
n=0
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

0
Epoch 1/10
6/6 [==============================] - 5s 784ms/step - loss: 1.2026 - accuracy: 0.6412 - val_loss: 0.3062 - val_accuracy: 0.8859
Epoch 2/10
6/6 [==============================] - 4s 819ms/step - loss: 0.9614 - accuracy: 0.7235 - val_loss: 0.3051 - val_accuracy: 0.8854
Epoch 3/10
6/6 [==============================] - 4s 801ms/step - loss: 0.7783 - accuracy: 0.7676 - val_loss: 0.3143 - val_accuracy: 0.8821
Epoch 4/10
6/6 [==============================] - 5s 1s/step - loss: 0.6864 - accuracy: 0.7765 - val_loss: 0.3210 - val_accuracy: 0.8804
Epoch 5/10
6/6 [==============================] - 5s 991ms/step - loss: 0.6753 - accuracy: 0.7941 - val_loss: 0.3565 - val_accuracy: 0.8713
Epoch 6/10
6/6 [==============================] - 5s 962ms/step - loss: 0.6469 - accuracy: 0.7971 - val_loss: 0.3252 - val_accuracy: 0.8825
Epoch 7/10
6/6 [==============================] - 5s 979ms/step - loss: 0.5440 - accuracy: 0.8235 - val_loss: 0.3618 - val_accuracy: 0.8686
Epoch 8/10
6/6 [=====

In [98]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

1
Epoch 1/10
11/11 [==============================] - 6s 460ms/step - loss: 1.0289 - accuracy: 0.6809 - val_loss: 0.2932 - val_accuracy: 0.8861
Epoch 2/10
11/11 [==============================] - 5s 458ms/step - loss: 0.8271 - accuracy: 0.7441 - val_loss: 0.3064 - val_accuracy: 0.8791
Epoch 3/10
11/11 [==============================] - 5s 457ms/step - loss: 0.6385 - accuracy: 0.7838 - val_loss: 0.3212 - val_accuracy: 0.8761
Epoch 4/10
11/11 [==============================] - 5s 453ms/step - loss: 0.5929 - accuracy: 0.8088 - val_loss: 0.3285 - val_accuracy: 0.8751
Epoch 5/10
11/11 [==============================] - 5s 454ms/step - loss: 0.5776 - accuracy: 0.8147 - val_loss: 0.3351 - val_accuracy: 0.8720
Epoch 6/10
11/11 [==============================] - 5s 452ms/step - loss: 0.5214 - accuracy: 0.8279 - val_loss: 0.3610 - val_accuracy: 0.8639
Epoch 7/10
11/11 [==============================] - 5s 503ms/step - loss: 0.4826 - accuracy: 0.8338 - val_loss: 0.3555 - val_accuracy: 0.8679
Epoc

In [99]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

2
Epoch 1/10
16/16 [==============================] - 6s 321ms/step - loss: 0.8779 - accuracy: 0.7333 - val_loss: 0.3092 - val_accuracy: 0.8769
Epoch 2/10
16/16 [==============================] - 5s 332ms/step - loss: 0.6635 - accuracy: 0.7961 - val_loss: 0.3198 - val_accuracy: 0.8759
Epoch 3/10
16/16 [==============================] - 5s 324ms/step - loss: 0.5763 - accuracy: 0.8078 - val_loss: 0.3258 - val_accuracy: 0.8784
Epoch 4/10
16/16 [==============================] - 5s 330ms/step - loss: 0.5436 - accuracy: 0.8265 - val_loss: 0.3267 - val_accuracy: 0.8748
Epoch 5/10
16/16 [==============================] - 5s 325ms/step - loss: 0.5007 - accuracy: 0.8490 - val_loss: 0.3317 - val_accuracy: 0.8798
Epoch 6/10
16/16 [==============================] - 5s 330ms/step - loss: 0.4573 - accuracy: 0.8480 - val_loss: 0.3504 - val_accuracy: 0.8734
Epoch 7/10
16/16 [==============================] - 5s 328ms/step - loss: 0.4174 - accuracy: 0.8490 - val_loss: 0.3290 - val_accuracy: 0.8780
Epoc

In [100]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

3
Epoch 1/10
22/22 [==============================] - 7s 268ms/step - loss: 0.7815 - accuracy: 0.7603 - val_loss: 0.3019 - val_accuracy: 0.8878
Epoch 2/10
22/22 [==============================] - 5s 255ms/step - loss: 0.5741 - accuracy: 0.8051 - val_loss: 0.3229 - val_accuracy: 0.8769
Epoch 3/10
22/22 [==============================] - 5s 255ms/step - loss: 0.5323 - accuracy: 0.8250 - val_loss: 0.3427 - val_accuracy: 0.8671
Epoch 4/10
22/22 [==============================] - 5s 257ms/step - loss: 0.5060 - accuracy: 0.8331 - val_loss: 0.3364 - val_accuracy: 0.8741
Epoch 5/10
22/22 [==============================] - 6s 261ms/step - loss: 0.4490 - accuracy: 0.8507 - val_loss: 0.3410 - val_accuracy: 0.8742
Epoch 6/10
22/22 [==============================] - 5s 256ms/step - loss: 0.4283 - accuracy: 0.8500 - val_loss: 0.3501 - val_accuracy: 0.8769
Epoch 7/10
22/22 [==============================] - 5s 256ms/step - loss: 0.3840 - accuracy: 0.8647 - val_loss: 0.3421 - val_accuracy: 0.8701
Epoc

In [101]:
#
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

4
Epoch 1/10
27/27 [==============================] - 7s 232ms/step - loss: 0.7094 - accuracy: 0.7729 - val_loss: 0.3204 - val_accuracy: 0.8786
Epoch 2/10
27/27 [==============================] - 6s 232ms/step - loss: 0.5403 - accuracy: 0.8229 - val_loss: 0.3303 - val_accuracy: 0.8781
Epoch 3/10
27/27 [==============================] - 6s 232ms/step - loss: 0.4803 - accuracy: 0.8471 - val_loss: 0.3282 - val_accuracy: 0.8771
Epoch 4/10
27/27 [==============================] - 6s 233ms/step - loss: 0.4393 - accuracy: 0.8488 - val_loss: 0.3412 - val_accuracy: 0.8719
Epoch 5/10
27/27 [==============================] - 6s 232ms/step - loss: 0.4117 - accuracy: 0.8612 - val_loss: 0.3527 - val_accuracy: 0.8734
Epoch 6/10
27/27 [==============================] - 6s 232ms/step - loss: 0.3707 - accuracy: 0.8694 - val_loss: 0.3449 - val_accuracy: 0.8746
Epoch 7/10
27/27 [==============================] - 6s 232ms/step - loss: 0.3607 - accuracy: 0.8776 - val_loss: 0.3441 - val_accuracy: 0.8741
Epoc

In [102]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

5
Epoch 1/10
32/32 [==============================] - 7s 208ms/step - loss: 0.6534 - accuracy: 0.7966 - val_loss: 0.3244 - val_accuracy: 0.8749
Epoch 2/10
32/32 [==============================] - 7s 210ms/step - loss: 0.4749 - accuracy: 0.8544 - val_loss: 0.3338 - val_accuracy: 0.8759
Epoch 3/10
32/32 [==============================] - 7s 209ms/step - loss: 0.4315 - accuracy: 0.8691 - val_loss: 0.3257 - val_accuracy: 0.8779
Epoch 4/10
32/32 [==============================] - 7s 209ms/step - loss: 0.3925 - accuracy: 0.8667 - val_loss: 0.3362 - val_accuracy: 0.8759
Epoch 5/10
32/32 [==============================] - 7s 212ms/step - loss: 0.3651 - accuracy: 0.8819 - val_loss: 0.3534 - val_accuracy: 0.8760
Epoch 6/10
32/32 [==============================] - 7s 208ms/step - loss: 0.3437 - accuracy: 0.8863 - val_loss: 0.3526 - val_accuracy: 0.8706
Epoch 7/10
32/32 [==============================] - 7s 209ms/step - loss: 0.3195 - accuracy: 0.8882 - val_loss: 0.3659 - val_accuracy: 0.8759
Epoc

In [103]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

6
Epoch 1/10
38/38 [==============================] - 8s 185ms/step - loss: 0.6110 - accuracy: 0.8227 - val_loss: 0.3177 - val_accuracy: 0.8789
Epoch 2/10
38/38 [==============================] - 7s 186ms/step - loss: 0.4464 - accuracy: 0.8584 - val_loss: 0.3254 - val_accuracy: 0.8836
Epoch 3/10
38/38 [==============================] - 7s 186ms/step - loss: 0.3820 - accuracy: 0.8744 - val_loss: 0.3335 - val_accuracy: 0.8757
Epoch 4/10
38/38 [==============================] - 7s 186ms/step - loss: 0.3498 - accuracy: 0.8857 - val_loss: 0.3406 - val_accuracy: 0.8723
Epoch 5/10
38/38 [==============================] - 7s 188ms/step - loss: 0.3152 - accuracy: 0.8929 - val_loss: 0.3355 - val_accuracy: 0.8761
Epoch 6/10
38/38 [==============================] - 7s 187ms/step - loss: 0.2980 - accuracy: 0.8983 - val_loss: 0.3364 - val_accuracy: 0.8829
Epoch 7/10
38/38 [==============================] - 7s 187ms/step - loss: 0.2791 - accuracy: 0.9042 - val_loss: 0.3946 - val_accuracy: 0.8751
Epoc

In [104]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

7
Epoch 1/10
43/43 [==============================] - 8s 173ms/step - loss: 0.5758 - accuracy: 0.8254 - val_loss: 0.3074 - val_accuracy: 0.8828
Epoch 2/10
43/43 [==============================] - 7s 174ms/step - loss: 0.4433 - accuracy: 0.8566 - val_loss: 0.3124 - val_accuracy: 0.8816
Epoch 3/10
43/43 [==============================] - 7s 173ms/step - loss: 0.3861 - accuracy: 0.8706 - val_loss: 0.3213 - val_accuracy: 0.8801
Epoch 4/10
43/43 [==============================] - 8s 186ms/step - loss: 0.3531 - accuracy: 0.8757 - val_loss: 0.3451 - val_accuracy: 0.8756
Epoch 5/10
43/43 [==============================] - 8s 180ms/step - loss: 0.3253 - accuracy: 0.8915 - val_loss: 0.3584 - val_accuracy: 0.8806
Epoch 6/10
43/43 [==============================] - 7s 172ms/step - loss: 0.3095 - accuracy: 0.8974 - val_loss: 0.3307 - val_accuracy: 0.8849
Epoch 7/10
43/43 [==============================] - 7s 174ms/step - loss: 0.2806 - accuracy: 0.9033 - val_loss: 0.3248 - val_accuracy: 0.8859
Epoc

In [105]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

8
Epoch 1/10
48/48 [==============================] - 11s 197ms/step - loss: 0.5339 - accuracy: 0.8297 - val_loss: 0.3120 - val_accuracy: 0.8814
Epoch 2/10
48/48 [==============================] - 9s 182ms/step - loss: 0.4252 - accuracy: 0.8644 - val_loss: 0.3137 - val_accuracy: 0.8821
Epoch 3/10
48/48 [==============================] - 8s 176ms/step - loss: 0.3877 - accuracy: 0.8722 - val_loss: 0.3233 - val_accuracy: 0.8811
Epoch 4/10
48/48 [==============================] - 8s 174ms/step - loss: 0.3387 - accuracy: 0.8889 - val_loss: 0.3141 - val_accuracy: 0.8841
Epoch 5/10
48/48 [==============================] - 8s 173ms/step - loss: 0.3062 - accuracy: 0.9029 - val_loss: 0.3430 - val_accuracy: 0.8784
Epoch 6/10
48/48 [==============================] - 8s 171ms/step - loss: 0.2960 - accuracy: 0.8993 - val_loss: 0.3292 - val_accuracy: 0.8875
Epoch 7/10
48/48 [==============================] - 8s 168ms/step - loss: 0.2756 - accuracy: 0.9105 - val_loss: 0.3373 - val_accuracy: 0.8828
Epo

In [106]:
#
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

9
Epoch 1/10
54/54 [==============================] - 9s 151ms/step - loss: 0.5194 - accuracy: 0.8376 - val_loss: 0.3125 - val_accuracy: 0.8812
Epoch 2/10
54/54 [==============================] - 10s 180ms/step - loss: 0.4057 - accuracy: 0.8715 - val_loss: 0.2956 - val_accuracy: 0.8919
Epoch 3/10
54/54 [==============================] - 9s 160ms/step - loss: 0.3567 - accuracy: 0.8812 - val_loss: 0.3408 - val_accuracy: 0.8735
Epoch 4/10
54/54 [==============================] - 8s 149ms/step - loss: 0.3378 - accuracy: 0.8906 - val_loss: 0.3236 - val_accuracy: 0.8839
Epoch 5/10
54/54 [==============================] - 8s 149ms/step - loss: 0.2954 - accuracy: 0.8956 - val_loss: 0.3279 - val_accuracy: 0.8845
Epoch 6/10
54/54 [==============================] - 8s 149ms/step - loss: 0.2782 - accuracy: 0.9056 - val_loss: 0.3271 - val_accuracy: 0.8864
Epoch 7/10
54/54 [==============================] - 8s 149ms/step - loss: 0.2733 - accuracy: 0.9109 - val_loss: 0.3422 - val_accuracy: 0.8834
Epo

In [107]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

10
Epoch 1/10
59/59 [==============================] - 9s 146ms/step - loss: 0.5287 - accuracy: 0.8361 - val_loss: 0.2996 - val_accuracy: 0.8882
Epoch 2/10
59/59 [==============================] - 9s 146ms/step - loss: 0.4182 - accuracy: 0.8623 - val_loss: 0.2946 - val_accuracy: 0.8869
Epoch 3/10
59/59 [==============================] - 8s 143ms/step - loss: 0.3545 - accuracy: 0.8834 - val_loss: 0.3075 - val_accuracy: 0.8894
Epoch 4/10
59/59 [==============================] - 8s 143ms/step - loss: 0.3359 - accuracy: 0.8874 - val_loss: 0.3092 - val_accuracy: 0.8856
Epoch 5/10
59/59 [==============================] - 8s 143ms/step - loss: 0.3165 - accuracy: 0.8939 - val_loss: 0.2918 - val_accuracy: 0.8900
Epoch 6/10
59/59 [==============================] - 8s 142ms/step - loss: 0.2933 - accuracy: 0.9008 - val_loss: 0.3336 - val_accuracy: 0.8824
Epoch 7/10
59/59 [==============================] - 8s 142ms/step - loss: 0.2702 - accuracy: 0.9070 - val_loss: 0.3142 - val_accuracy: 0.8861
Epo

In [108]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

11
Epoch 1/10
64/64 [==============================] - 10s 138ms/step - loss: 0.4959 - accuracy: 0.8417 - val_loss: 0.3099 - val_accuracy: 0.8813
Epoch 2/10
64/64 [==============================] - 9s 140ms/step - loss: 0.3828 - accuracy: 0.8728 - val_loss: 0.2862 - val_accuracy: 0.8861
Epoch 3/10
64/64 [==============================] - 9s 141ms/step - loss: 0.3495 - accuracy: 0.8848 - val_loss: 0.2970 - val_accuracy: 0.8824
Epoch 4/10
64/64 [==============================] - 9s 138ms/step - loss: 0.3251 - accuracy: 0.8939 - val_loss: 0.2944 - val_accuracy: 0.8877
Epoch 5/10
64/64 [==============================] - 9s 141ms/step - loss: 0.3035 - accuracy: 0.9037 - val_loss: 0.3106 - val_accuracy: 0.8874
Epoch 6/10
64/64 [==============================] - 9s 137ms/step - loss: 0.2804 - accuracy: 0.9108 - val_loss: 0.3104 - val_accuracy: 0.8867
Epoch 7/10
64/64 [==============================] - 9s 141ms/step - loss: 0.2543 - accuracy: 0.9174 - val_loss: 0.3231 - val_accuracy: 0.8861
Ep

In [109]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

12
Epoch 1/10
70/70 [==============================] - 10s 133ms/step - loss: 0.4833 - accuracy: 0.8423 - val_loss: 0.2963 - val_accuracy: 0.8859
Epoch 2/10
70/70 [==============================] - 9s 133ms/step - loss: 0.3741 - accuracy: 0.8688 - val_loss: 0.3125 - val_accuracy: 0.8819
Epoch 3/10
70/70 [==============================] - 9s 135ms/step - loss: 0.3420 - accuracy: 0.8921 - val_loss: 0.3054 - val_accuracy: 0.8869
Epoch 4/10
70/70 [==============================] - 9s 133ms/step - loss: 0.3158 - accuracy: 0.8989 - val_loss: 0.3168 - val_accuracy: 0.8851
Epoch 5/10
70/70 [==============================] - 9s 133ms/step - loss: 0.2906 - accuracy: 0.9045 - val_loss: 0.3167 - val_accuracy: 0.8849
Epoch 6/10
70/70 [==============================] - 9s 134ms/step - loss: 0.2774 - accuracy: 0.9070 - val_loss: 0.2988 - val_accuracy: 0.8891
Epoch 7/10
70/70 [==============================] - 9s 135ms/step - loss: 0.2482 - accuracy: 0.9188 - val_loss: 0.3315 - val_accuracy: 0.8896
Ep

In [110]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

13
Epoch 1/10
75/75 [==============================] - 10s 128ms/step - loss: 0.4972 - accuracy: 0.8433 - val_loss: 0.2963 - val_accuracy: 0.8822
Epoch 2/10
75/75 [==============================] - 10s 130ms/step - loss: 0.3674 - accuracy: 0.8805 - val_loss: 0.2988 - val_accuracy: 0.8844
Epoch 3/10
75/75 [==============================] - 10s 129ms/step - loss: 0.3502 - accuracy: 0.8878 - val_loss: 0.2893 - val_accuracy: 0.8917
Epoch 4/10
75/75 [==============================] - 10s 130ms/step - loss: 0.3021 - accuracy: 0.8983 - val_loss: 0.3170 - val_accuracy: 0.8839
Epoch 5/10
75/75 [==============================] - 10s 130ms/step - loss: 0.2788 - accuracy: 0.9088 - val_loss: 0.3149 - val_accuracy: 0.8855
Epoch 6/10
75/75 [==============================] - 10s 130ms/step - loss: 0.2674 - accuracy: 0.9105 - val_loss: 0.3185 - val_accuracy: 0.8859
Epoch 7/10
75/75 [==============================] - 10s 131ms/step - loss: 0.2494 - accuracy: 0.9179 - val_loss: 0.3236 - val_accuracy: 0.8

In [111]:
#
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

14
Epoch 1/10
80/80 [==============================] - 11s 124ms/step - loss: 0.4921 - accuracy: 0.8445 - val_loss: 0.3000 - val_accuracy: 0.8866
Epoch 2/10
80/80 [==============================] - 10s 127ms/step - loss: 0.3789 - accuracy: 0.8725 - val_loss: 0.2950 - val_accuracy: 0.8879
Epoch 3/10
80/80 [==============================] - 10s 126ms/step - loss: 0.3454 - accuracy: 0.8825 - val_loss: 0.2890 - val_accuracy: 0.8897
Epoch 4/10
80/80 [==============================] - 10s 129ms/step - loss: 0.3145 - accuracy: 0.8906 - val_loss: 0.2968 - val_accuracy: 0.8896
Epoch 5/10
80/80 [==============================] - 10s 126ms/step - loss: 0.2884 - accuracy: 0.9029 - val_loss: 0.3016 - val_accuracy: 0.8917
Epoch 6/10
80/80 [==============================] - 10s 126ms/step - loss: 0.2732 - accuracy: 0.9076 - val_loss: 0.3164 - val_accuracy: 0.8886
Epoch 7/10
80/80 [==============================] - 10s 126ms/step - loss: 0.2533 - accuracy: 0.9176 - val_loss: 0.3019 - val_accuracy: 0.8

In [112]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

15
Epoch 1/10
85/85 [==============================] - 11s 123ms/step - loss: 0.4680 - accuracy: 0.8436 - val_loss: 0.2869 - val_accuracy: 0.8922
Epoch 2/10
85/85 [==============================] - 10s 124ms/step - loss: 0.3828 - accuracy: 0.8765 - val_loss: 0.2853 - val_accuracy: 0.8917
Epoch 3/10
85/85 [==============================] - 10s 122ms/step - loss: 0.3441 - accuracy: 0.8864 - val_loss: 0.3068 - val_accuracy: 0.8861
Epoch 4/10
85/85 [==============================] - 10s 122ms/step - loss: 0.3066 - accuracy: 0.8919 - val_loss: 0.3233 - val_accuracy: 0.8910
Epoch 5/10
85/85 [==============================] - 10s 121ms/step - loss: 0.2890 - accuracy: 0.9006 - val_loss: 0.3056 - val_accuracy: 0.8904
Epoch 6/10
85/85 [==============================] - 10s 122ms/step - loss: 0.2704 - accuracy: 0.9134 - val_loss: 0.2974 - val_accuracy: 0.8934
Epoch 7/10
85/85 [==============================] - 10s 124ms/step - loss: 0.2504 - accuracy: 0.9154 - val_loss: 0.3319 - val_accuracy: 0.8

In [113]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

16
Epoch 1/10
91/91 [==============================] - 12s 121ms/step - loss: 0.4930 - accuracy: 0.8343 - val_loss: 0.2826 - val_accuracy: 0.8916
Epoch 2/10
91/91 [==============================] - 11s 121ms/step - loss: 0.3861 - accuracy: 0.8656 - val_loss: 0.2813 - val_accuracy: 0.8954
Epoch 3/10
91/91 [==============================] - 11s 122ms/step - loss: 0.3633 - accuracy: 0.8780 - val_loss: 0.2931 - val_accuracy: 0.8932
Epoch 4/10
91/91 [==============================] - 11s 121ms/step - loss: 0.3234 - accuracy: 0.8860 - val_loss: 0.2896 - val_accuracy: 0.8939
Epoch 5/10
91/91 [==============================] - 11s 121ms/step - loss: 0.2995 - accuracy: 0.8983 - val_loss: 0.2976 - val_accuracy: 0.8969
Epoch 6/10
91/91 [==============================] - 11s 121ms/step - loss: 0.2871 - accuracy: 0.9054 - val_loss: 0.3167 - val_accuracy: 0.8912
Epoch 7/10
91/91 [==============================] - 11s 122ms/step - loss: 0.2671 - accuracy: 0.9121 - val_loss: 0.3068 - val_accuracy: 0.8

In [114]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

17
Epoch 1/10
96/96 [==============================] - 12s 120ms/step - loss: 0.5223 - accuracy: 0.8242 - val_loss: 0.2856 - val_accuracy: 0.8888
Epoch 2/10
96/96 [==============================] - 11s 120ms/step - loss: 0.3996 - accuracy: 0.8636 - val_loss: 0.2836 - val_accuracy: 0.8919
Epoch 3/10
96/96 [==============================] - 12s 121ms/step - loss: 0.3770 - accuracy: 0.8708 - val_loss: 0.2845 - val_accuracy: 0.8926
Epoch 4/10
96/96 [==============================] - 11s 120ms/step - loss: 0.3494 - accuracy: 0.8802 - val_loss: 0.3031 - val_accuracy: 0.8929
Epoch 5/10
96/96 [==============================] - 11s 119ms/step - loss: 0.3122 - accuracy: 0.8938 - val_loss: 0.2950 - val_accuracy: 0.8976
Epoch 6/10
96/96 [==============================] - 12s 123ms/step - loss: 0.2835 - accuracy: 0.9015 - val_loss: 0.3047 - val_accuracy: 0.8945
Epoch 7/10
96/96 [==============================] - 11s 115ms/step - loss: 0.2690 - accuracy: 0.9083 - val_loss: 0.3236 - val_accuracy: 0.8

In [115]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

18
Epoch 1/10
101/101 [==============================] - 13s 117ms/step - loss: 0.5424 - accuracy: 0.8116 - val_loss: 0.2820 - val_accuracy: 0.8951
Epoch 2/10
101/101 [==============================] - 12s 116ms/step - loss: 0.4285 - accuracy: 0.8511 - val_loss: 0.2774 - val_accuracy: 0.8961
Epoch 3/10
101/101 [==============================] - 12s 118ms/step - loss: 0.3865 - accuracy: 0.8625 - val_loss: 0.2842 - val_accuracy: 0.8941
Epoch 4/10
101/101 [==============================] - 12s 118ms/step - loss: 0.3507 - accuracy: 0.8763 - val_loss: 0.2794 - val_accuracy: 0.8997
Epoch 5/10
101/101 [==============================] - 12s 117ms/step - loss: 0.3359 - accuracy: 0.8845 - val_loss: 0.2883 - val_accuracy: 0.8965
Epoch 6/10
101/101 [==============================] - 12s 116ms/step - loss: 0.3007 - accuracy: 0.8958 - val_loss: 0.3086 - val_accuracy: 0.8971
Epoch 7/10
101/101 [==============================] - 12s 116ms/step - loss: 0.2938 - accuracy: 0.8983 - val_loss: 0.2851 - val

In [116]:
#
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

19
Epoch 1/10
110/110 [==============================] - 13s 113ms/step - loss: 0.5562 - accuracy: 0.8055 - val_loss: 0.2834 - val_accuracy: 0.8956
Epoch 2/10
110/110 [==============================] - 13s 116ms/step - loss: 0.4476 - accuracy: 0.8364 - val_loss: 0.2711 - val_accuracy: 0.8997
Epoch 3/10
110/110 [==============================] - 13s 115ms/step - loss: 0.4114 - accuracy: 0.8544 - val_loss: 0.2884 - val_accuracy: 0.8974
Epoch 4/10
110/110 [==============================] - 12s 112ms/step - loss: 0.3829 - accuracy: 0.8654 - val_loss: 0.2840 - val_accuracy: 0.9001
Epoch 5/10
110/110 [==============================] - 12s 114ms/step - loss: 0.3488 - accuracy: 0.8758 - val_loss: 0.2967 - val_accuracy: 0.8961
Epoch 6/10
110/110 [==============================] - 12s 112ms/step - loss: 0.3344 - accuracy: 0.8847 - val_loss: 0.2898 - val_accuracy: 0.9000
Epoch 7/10
110/110 [==============================] - 12s 113ms/step - loss: 0.3067 - accuracy: 0.8976 - val_loss: 0.2933 - val

In [117]:
#new_model_nc_dir  = "C:/Users/fjdurlop/Documents/upc/models/"+dataset+"/C3/"+dataset+"_model_c3_aug_nc_e1"
new_model_nc_dir  = "D:/models/aug_22/"+dataset+"/C3/"+dataset+"_model_c3_aug_nc_e1"
i=0

for model in models_nc:
    model.save(new_model_nc_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_aug_nc_e1_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_aug_nc_e1_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_aug_nc_e1_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_aug_nc_e1_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_aug_nc_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_aug_nc_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_aug_nc_e1_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C3/fashion_model_c3_aug_nc_e1_7\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/mo

In [118]:
try:
    del nc_values
    del top_images_by_nc
    del top_labels_by_nc
    del image_sets_nc
    del label_sets_nc
    del models_nc
except:
    print("Error")

In [119]:
gc.collect()

372064